# 0188  Hard  188. Best Time to Buy and Sell Stock IV

In [ ]:
class Solution:
    def maxProfit(self, k, prices):
        if k >= len(prices) // 2: return sum(sell - buy for sell, buy in zip(prices[1:], prices[:-1]) if sell - buy > 0)
        dp = [[0, -float("inf")] for _ in range(k + 1)]
        for p in prices:
            for i in range(k + 1):
                if i and dp[i - 1][1] + p > dp[i][0]: dp[i][0] = dp[i - 1][1] + p 
                if dp[i][0] - p > dp[i][1]: dp[i][1] = dp[i][0] - p
        return dp[-1][0]

# 0188 Hard 188 Best Time to Buy and Sell Stock IV

In [ ]:
# Time:  O(n) on average, using Median of Medians could achieve O(n) (Intro Select)
# Space: O(n)

import random


class Solution(object):
    def maxProfit(self, k, prices):
        """
        :type k: int
        :type prices: List[int]
        :rtype: int
        """
        def nth_element(nums, n, compare=lambda a, b: a < b):
            def tri_partition(nums, left, right, target, compare):
                mid = left
                while mid <= right:
                    if nums[mid] == target:
                        mid += 1
                    elif compare(nums[mid], target):
                        nums[left], nums[mid] = nums[mid], nums[left]
                        left += 1
                        mid += 1
                    else:
                        nums[mid], nums[right] = nums[right], nums[mid]
                        right -= 1
                return left, right

            left, right = 0, len(nums)-1
            while left <= right:
                pivot_idx = random.randint(left, right)
                pivot_left, pivot_right = tri_partition(nums, left, right, nums[pivot_idx], compare)
                if pivot_left <= n <= pivot_right:
                    return
                elif pivot_left > n:
                    right = pivot_left-1
                else:  # pivot_right < n.
                    left = pivot_right+1

        profits = []
        v_p_stk = []  # mono stack, where v is increasing and p is strictly decreasing
        v, p = -1, -1
        while p+1 < len(prices): # at most O(n) peaks, so v_p_stk and profits are both at most O(n) space
            for v in xrange(p+1, len(prices)-1):
                if prices[v] < prices[v+1]:
                    break
            else:
                v = len(prices)-1
            for p in xrange(v, len(prices)-1):
                if prices[p] > prices[p+1]:
                    break 
            else:
                p = len(prices)-1
            while v_p_stk and prices[v_p_stk[-1][0]] > prices[v]:  # not overlapped
                last_v, last_p = v_p_stk.pop()
                profits.append(prices[last_p]-prices[last_v])  # count [prices[last_v], prices[last_p]] interval
            while v_p_stk and prices[v_p_stk[-1][1]] <= prices[p]:  # overlapped
                # prices[last_v] <= prices[v] <= prices[last_p] <= prices[p],
                # treat overlapped as [prices[v], prices[last_p]], [prices[last_v], prices[p]] intervals due to invariant max profit
                last_v, last_p = v_p_stk.pop()
                profits.append(prices[last_p]-prices[v])  # count [prices[v], prices[last_p]] interval
                v = last_v
            v_p_stk.append((v, p))  # keep [prices[last_v], prices[p]] interval to check overlapped      
        while v_p_stk:
            last_v, last_p = v_p_stk.pop()
            profits.append(prices[last_p]-prices[last_v])  # count [prices[last_v], prices[last_p]] interval
        if k > len(profits):
            k = len(profits)
        else:
            nth_element(profits, k-1, compare=lambda a, b: a > b)
        return sum(profits[i] for i in xrange(k))  # top k profits of nonoverlapped intervals


# Time:  O(k * n)
# Space: O(k)
class Solution2(object):
    def maxProfit(self, k, prices):
        """
        :type k: int
        :type prices: List[int]
        :rtype: int
        """
        def maxAtMostNPairsProfit(sprices):
            profit = 0
            for i in xrange(len(prices) - 1):
                profit += max(0, prices[i + 1] - prices[i])
            return profit

        def maxAtMostKPairsProfit(prices, k):
            max_buy = [float("-inf") for _ in xrange(k + 1)]
            max_sell = [0 for _ in xrange(k + 1)]
            for i in xrange(len(prices)):
                for j in xrange(1, k + 1):
                    max_buy[j] = max(max_buy[j], max_sell[j-1] - prices[i])
                    max_sell[j] = max(max_sell[j], max_buy[j] + prices[i])
            return max_sell[k]

        if k >= len(prices) // 2:
            return maxAtMostNPairsProfit(prices)

        return maxAtMostKPairsProfit(prices, k)

# 0212  Hard  212. Word Search II

In [ ]:
class Solution:
    def findWords(self, board, words):
        def explore(i, j, cur):
            visited[i][j] = 0
            if "#" in cur and cur["#"] not in res_set: res.append(cur["#"]); res_set.add(cur["#"])
            for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
                if 0 <= x < m and 0 <= y < n and board[x][y] in cur and visited[x][y] == -1: explore(x, y, trie[cur[board[x][y]]])
            visited[i][j] = -1
        trie, cnt, m, n, res, res_set = {}, 1, len(board), len(board and board[0]), [], set()
        visited, trie[0] = [[-1] * n for _ in range(m)], {}
        for w in words:
            cur = trie[0]
            for c in w:
                if c not in cur: trie[cnt], cur[c], cnt = {}, cnt, cnt + 1
                cur = trie[cur[c]]
            cur["#"] = w
        for i in range(m):
            for j in range(n):
                if board[i][j] in trie[0]: explore(i, j, trie[trie[0][board[i][j]]])
        return res

# 0212 Hard 212 Word Search II

In [ ]:
# TLE
class Solution(object):
    def findWords(self, board, words):
        def getNeighbor(i, j):
            opt = []
            if i+1<M: opt.append((i+1, j))
            if j+1<N: opt.append((i, j+1))
            if i-1>=0: opt.append((i-1, j))
            if j-1>=0: opt.append((i, j-1))
            return opt

        def dfs(i, j, l, word):
            if word in found: return
            if board[i][j]!=word[l]: return

            if l==len(word)-1 and board[i][j]==word[l]:
                opt.append(word)
                found.add(word)
                return

            char = board[i][j]
            board[i][j] = '#'

            for ni, nj in getNeighbor(i, j):
                dfs(ni, nj, l+1, word)

            board[i][j] = char

        opt = []
        M = len(board)
        N = len(board[0])
        found = set()

        for i in xrange(M):
            for j in xrange(N):
                for word in words:
                    if word not in found:
                        dfs(i, j, 0, word)
        return opt


board = [
  ['o','a','a','n'],
  ['e','t','a','e'],
  ['i','h','k','r'],
  ['i','f','l','v']
]
words = ["oath","pea","eat","rain"]

print Solution().findWords(board, words)




class Solution(object):
    def findWords(self, board, words):
        def dfs(i, j, parent):
            c = board[i][j]
            node = parent[c]
            
            if '.' in node: ans.append(node.pop('.'))
            
            board[i][j] = '#'
            
            for (rowOffset, colOffset) in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
                newRow, newCol = i + rowOffset, j + colOffset     
                if newRow<0 or newRow>=len(board) or newCol<0 or newCol>=len(board[0]): continue
                if not board[newRow][newCol] in node: continue
                dfs(newRow, newCol, node)
            
            board[i][j] = c
            if not node: parent.pop(c)
                
        trie = {}
        ans = []
        
        #build trie
        for word in words:
            node = trie
            for c in word:
                if c not in node:
                    node[c] = {}
                node = node[c]
            node['.'] = word
        print trie
        
        for i in xrange(len(board)):
            for j in xrange(len(board[0])):
                if board[i][j] in trie: dfs(i, j, trie)
        
        
        return ans
                    
            
                
        

# 0214  Hard  214. Shortest Palindrome

In [ ]:
class Solution:
    def shortestPalindrome(self, s, pre = ""):
        for i in range(1, len(s) // 2 + 2):
            if s[i - 1:].startswith(s[:i][::-1]): pre = s[2* i - 1:][::-1]
            if s[i:].startswith(s[:i][::-1]): pre = s[2* i:][::-1]
        return pre + s

# 0214 Hard 214 Shortest Palindrome

In [ ]:
# Time:  O(n)
# Space: O(n)

# optimized from Solution2
class Solution(object):
    def shortestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        def getPrefix(pattern):
            prefix = [-1] * len(pattern)
            j = -1
            for i in xrange(1, len(pattern)):
                while j > -1 and pattern[j+1] != pattern[i]:
                    j = prefix[j]
                if pattern[j+1] == pattern[i]:
                    j += 1
                prefix[i] = j
            return prefix

        if not s:
            return s

        A = s + '#' + s[::-1]
        return s[getPrefix(A)[-1]+1:][::-1] + s


# Time:  O(n)
# Space: O(n)
class Solution2(object):
    def shortestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        def getPrefix(pattern):
            prefix = [-1] * len(pattern)
            j = -1
            for i in xrange(1, len(pattern)):
                while j > -1 and pattern[j+1] != pattern[i]:
                    j = prefix[j]
                if pattern[j+1] == pattern[i]:
                    j += 1
                prefix[i] = j
            return prefix

        if not s:
            return s

        A = s + s[::-1]
        prefix = getPrefix(A)
        i = prefix[-1]
        while i >= len(s):
            i = prefix[i]
        return s[i+1:][::-1] + s


# Time:  O(n)
# Space: O(n)
# Manacher's Algorithm
class Solution3(object):
    def shortestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        def preProcess(s):
            if not s:
                return ['^', '$']
            string = ['^']
            for c in s:
                string +=  ['#', c]
            string += ['#', '$']
            return string

        string = preProcess(s)
        palindrome = [0] * len(string)
        center, right = 0, 0
        for i in xrange(1, len(string) - 1):
            i_mirror = 2 * center - i
            if right > i:
                palindrome[i] = min(right - i, palindrome[i_mirror])
            else:
                palindrome[i] = 0

            while string[i + 1 + palindrome[i]] == string[i - 1 - palindrome[i]]:
                palindrome[i] += 1

            if i + palindrome[i] > right:
                center, right = i, i + palindrome[i]

        max_len = 0
        for i in xrange(1, len(string) - 1):
            if i - palindrome[i] == 1:
                max_len = palindrome[i]
        return s[len(s)-1:max_len-1:-1] + s

# 0218  Hard  218. The Skyline Problem

In [ ]:
class Solution:
    def getSkyline(self, buildings):
        events = sorted([(L, -H, R) for L, R, H in buildings] + list({(R, 0, None) for _, R, _ in buildings}))
        res, hp = [[0, 0]], [(0, float("inf"))]
        for x, negH, R in events:
            while x >= hp[0][1]: 
                heapq.heappop(hp)
            if negH: 
                heapq.heappush(hp, (negH, R))
            if res[-1][1] + hp[0][0]: 
                res += [x, -hp[0][0]],
        return res[1:]

# 0218 Hard 218 The Skyline Problem

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

start, end, height = 0, 1, 2
class Solution(object):
    # @param {integer[][]} buildings
    # @return {integer[][]}
    def getSkyline(self, buildings):
        intervals = self.ComputeSkylineInInterval(buildings, 0, len(buildings))

        res = []
        last_end = -1
        for interval in intervals:
            if last_end != -1 and last_end < interval[start]:
                res.append([last_end, 0])
            res.append([interval[start], interval[height]])
            last_end = interval[end]
        if last_end != -1:
            res.append([last_end, 0])

        return res

    # Divide and Conquer.
    def ComputeSkylineInInterval(self, buildings, left_endpoint, right_endpoint):
        if right_endpoint - left_endpoint <= 1:
            return buildings[left_endpoint:right_endpoint]
        mid = left_endpoint + ((right_endpoint - left_endpoint) / 2)
        left_skyline = self.ComputeSkylineInInterval(buildings, left_endpoint, mid)
        right_skyline = self.ComputeSkylineInInterval(buildings, mid, right_endpoint)
        return self.MergeSkylines(left_skyline, right_skyline)

    # Merge Sort.
    def MergeSkylines(self, left_skyline, right_skyline):
        i, j = 0, 0
        merged = []

        while i < len(left_skyline) and j < len(right_skyline):
            if left_skyline[i][end] < right_skyline[j][start]:
                merged.append(left_skyline[i])
                i += 1
            elif right_skyline[j][end] < left_skyline[i][start]:
                merged.append(right_skyline[j])
                j += 1
            elif left_skyline[i][start] <= right_skyline[j][start]:
                i, j = self.MergeIntersectSkylines(merged, left_skyline[i], i,\
                                                   right_skyline[j], j)
            else: # left_skyline[i][start] > right_skyline[j][start].
                j, i = self.MergeIntersectSkylines(merged, right_skyline[j], j, \
                                                   left_skyline[i], i)

        # Insert the remaining skylines.
        merged += left_skyline[i:]
        merged += right_skyline[j:]
        return merged

    # a[start] <= b[start]
    def MergeIntersectSkylines(self, merged, a, a_idx, b, b_idx):
        if a[end] <= b[end]:
            if a[height] > b[height]:   # |aaa|
                if b[end] != a[end]:    # |abb|b
                    b[start] = a[end]
                    merged.append(a)
                    a_idx += 1
                else:             # aaa
                    b_idx += 1    # abb
            elif a[height] == b[height]:  # abb
                b[start] = a[start]       # abb
                a_idx += 1
            else:  # a[height] < b[height].
                if a[start] != b[start]:                            #    bb
                    merged.append([a[start], b[start], a[height]])  # |a|bb
                a_idx += 1
        else:  # a[end] > b[end].
            if a[height] >= b[height]:  # aaaa
                b_idx += 1              # abba
            else:
                #    |bb|
                # |a||bb|a
                if a[start] != b[start]:
                    merged.append([a[start], b[start], a[height]])
                a[start] = b[end]
                merged.append(b)
                b_idx += 1
        return a_idx, b_idx

# 0220  Hard  220. Contains Duplicate III

In [ ]:
class Solution:
    def containsNearbyAlmostDuplicate(self, nums, k, t):
        if t < 0: return False
        d = {}
        for i in range(len(nums)):
            m = nums[i] // (t + 1)
            if m in d or (m - 1 in d and nums[i] - d[m - 1] <= t) or (m + 1 in d and d[m + 1] - nums[i] <= t):
                return True
            d[m] = nums[i]
            if i >= k: del d[nums[i - k] // (t + 1)]
        return False

# 0220 Hard 220. Contains Duplicate III

In [ ]:
from collections import OrderedDict
class Solution(object):
    def containsNearbyAlmostDuplicate(self, nums, k, t):
        """
        :type nums: List[int]
        :type k: int
        :type t: int
        :rtype: bool
        """
        # https://discuss.leetcode.com/topic/19991/o-n-python-using-buckets-with-explanation-10-lines
        # Bucket sort. Each bucket has size of t. For each number, the possible
        # candidate can only be in the same bucket or the two buckets besides.
        # Keep as many as k buckets to ensure that the difference is at most k.
        buckets = {}
        for i, v in enumerate(nums):
            # t == 0 is a special case where we only have to check the bucket
            # that v is in.
            bucketNum, offset = (v / t, 1) if t else (v, 0)
            for idx in xrange(bucketNum - offset, bucketNum + offset + 1):
                if idx in buckets and abs(buckets[idx] - nums[i]) <= t:
                    return True

            buckets[bucketNum] = nums[i]
            if len(buckets) > k:
                # Remove the bucket which is too far away. Beware of zero t.
                del buckets[nums[i - k] / t if t else nums[i - k]]

        return False

# 0220 Hard 220 Contains Duplicate III

In [ ]:
"""
Time: O(N)
Space: O(K)

Imagine there are several different buckets. Each bucket can only contain one number.
Bucket1 contains a number within [0, t]
Bucket2 contains a number within [t+1, 2t+1]
Bucket3 contains a number within [2t+2, 3t+2]
...

For each num, put it in the bucket.
Before that.
If the bucket is already occupied, there must be another num that "abs(anotherNum - num) <= t", return True.
Check the neighbor buckets, the number within those might also "abs(anotherNum - num) <= t". If so, return True.

We only consider k numbers before num.
So remove nums[i-k-1] from the bucket if we already have k elements in the buckets.
"""
class Solution(object):
    def containsNearbyAlmostDuplicate(self, nums, k, t):
        bucket = {}
        
        for i, num in enumerate(nums):
            if len(bucket)>k:
                bidToRemove = nums[i-k-1]//(t+1)
                del bucket[bidToRemove]
            
            bid = num//(t+1)
            if bid in bucket: return True
            if bid+1 in bucket and abs(bucket[bid+1]-num)<=t: return True
            if bid-1 in bucket and abs(bucket[bid-1]-num)<=t: return True
            bucket[bid] = num
        
        return False

# 0224  Hard  224. Basic Calculator

In [ ]:
class Solution:
    def calculate(self, s):
        def calc(n2, op, n1): 
            return n1 + n2 if op == "+" else n1 - n2
        stack, i, num = [], 0, 0
        while i < len(s):
            j = i
            while j < len(s) and s[j].isdigit():
                num, j = num * 10 + int(s[j]), j + 1
            if i != j:
                stack.append(calc(num, stack.pop(), stack.pop()) if stack and s[i - 1] != "(" else num)
                num, j = 0, j - 1
            elif s[i] in "+-":
                stack.append(s[i])
            elif s[i] == ")" and len(stack) > 1:
                stack.append(calc(stack.pop(), stack.pop(), stack.pop()))
            i = j + 1
        return stack.pop()

# 0224 Hard 224 Basic Calculator

In [ ]:
# Time:  O(n)
# Space: O(n)

import operator


class Solution(object):
    def calculate(self, s):
        """
        :type s: str
        :rtype: int
        """
        def compute(operands, operators):
            right, left = operands.pop(), operands.pop()
            operands.append(ops[operators.pop()](left, right))

        ops = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.div}
        precedence = {'+':0, '-':0, '*':1, '/':1}
        operands, operators, operand = [], [], 0
        for i in xrange(len(s)):
            if s[i].isdigit():
                operand = operand*10 + int(s[i])
                if i == len(s)-1 or not s[i+1].isdigit():
                    operands.append(operand)
                    operand = 0
            elif s[i] == '(':
                operators.append(s[i])
            elif s[i] == ')':
                while operators[-1] != '(':
                    compute(operands, operators)
                operators.pop()
            elif s[i] in precedence:
                while operators and operators[-1] in precedence and \
                      precedence[operators[-1]] >= precedence[s[i]]:
                    compute(operands, operators)
                operators.append(s[i])
        while operators:
            compute(operands, operators)
        return operands[-1]


# Time:  O(n)
# Space: O(n)
class Solution2(object):
    # @param {string} s
    # @return {integer}
    def calculate(self, s):
        operands, operators = [], []
        operand = ""
        for i in reversed(xrange(len(s))):
            if s[i].isdigit():
                operand += s[i]
                if i == 0 or not s[i-1].isdigit():
                    operands.append(int(operand[::-1]))
                    operand = ""
            elif s[i] == ')' or s[i] == '+' or s[i] == '-':
                operators.append(s[i])
            elif s[i] == '(':
                while operators[-1] != ')':
                    self.compute(operands, operators)
                operators.pop()

        while operators:
            self.compute(operands, operators)

        return operands[-1]

    def compute(self, operands, operators):
        left, right = operands.pop(), operands.pop()
        op = operators.pop()
        if op == '+':
            operands.append(left + right)
        elif op == '-':
            operands.append(left - right)

# 0233  Hard  233. Number of Digit One

In [ ]:
class Solution:
    def countDigitOne(self, n):
        if n <= 0:
            return 0
        q, x, ans = n, 1, 0
        while q > 0:
            digit = q % 10
            q //= 10
            ans += q * x
            if digit == 1:
                ans += n % x + 1
            elif digit > 1:
                ans += x
            x *= 10
        return ans

# 0233 Hard 233 Number of Digit One

In [ ]:
# Time:  O(logn)
# Space: O(1)

class Solution(object):
    def countDigitOne(self, n):
        """
        :type n: int
        :rtype: int
        """
        DIGIT = 1
        is_zero = int(DIGIT == 0)
        result = is_zero
        base = 1
        while n >= base:
            result += (n//(10*base)-is_zero)*base + \
                      min(base, max(n%(10*base) - DIGIT*base + 1, 0))
            base *= 10
        return result

# 0239  Hard  239. Sliding Window Maximum

In [ ]:
class Solution:
    def maxSlidingWindow(self, nums, k):
        cnt, heap, res = collections.Counter(), [], []
        for i, num in enumerate(nums):
            heapq.heappush(heap, -num)
            cnt[num] += 1
            while not cnt[-heap[0]]:
                heapq.heappop(heap)
            if i >= k - 1:
                res.append(-heap[0])
                cnt[nums[i - k + 1]] -= 1
        return res

# 0239 Hard 239 Sliding Window Maximum

In [ ]:
class Solution(object):
    def maxSlidingWindow(self, nums, k):
        opt = []
        q = collections.deque()
        for i in xrange(len(nums)):
            n = nums[i]
            
            #move the window
            if q and q[0]<=i-k: q.popleft()

            #pop the right if the element in queue is not greater than the in-coming one
            #by doing this, we can always keep the max in the current window at left most
            while q and nums[q[-1]]<=n: q.pop()

            q.append(i)

            #add the max to the output array after the first kth element
            if 1+i>=k: opt.append(nums[q[0]])
        return opt


"""
Keep the element in sliding window in a heap, so we can get the max easily.
But we do not know whether which element in the heap is out of the window.
So we use a counter to track the count of the elements inside the window.

Time: O(NLogK), the `heapq.heappop(h)` in the while-loop in the for-loop will only be execute once for each element.
Space: O(N)
"""
class Solution(object):
    def maxSlidingWindow(self, nums, k):
        if not nums or not k: return []
        if k==1: return nums
        
        ans = []
        
        counter = collections.Counter(nums[:k])
        h = [-num for num in nums[:k]]
        heapq.heapify(h)
        
        for i in xrange(len(nums)):    
            while counter[-h[0]]==0: heapq.heappop(h)
            ans.append(-h[0])
            
            if i+k>=len(nums): break
            counter[nums[i]] -= 1
            heapq.heappush(h, -nums[i+k])
            counter[nums[i+k]] += 1
        
        return ans

"""
Use deque the slide the window through nums.
The deque will at most contain k elements.
Each time add the max element to ans.

Time: O(N), the `q.pop()` in the while-loop in the for-loop will only execute once for each element.
Space: O(N)
"""
class Solution(object):
    def maxSlidingWindow(self, nums, k):
        ans = []
        q = collections.deque()
        
        for i, num in enumerate(nums):
            #keep only the element in the window inside deque.
            #the window lies between index i-k+1 to i.
            if q and q[0][1]<=i-k: q.popleft()
            
            #keep the element in deque in descending order by popping the elements from rihgt that are smaller than num.
            #by keeping the deque in descending order, we can get the largest element by just getting the first element.
            #also since they are certainly not the max in the deque, we don't care.
            while q and q[-1][0]<=num: q.pop()
            q.append((num, i))
            
            #wait until i>=k-1 when the window pass at least k elements.
            if (i>=k-1): ans.append(q[0][0])
                
        return ans

# 0248  Hard  248. Strobogrammatic Number III

In [ ]:
class Solution:
    def strobogrammaticInRange(self, low, high):
        q, cnt, low, high, ln = ["", "0", "1", "8"], 0, int(low), int(high), len(high)
        while q:
            s = q.pop()
            if s and s[0] != "0" and low <= int(s) <= high: cnt += 1
            q += [l + s + r for l, r in (("8", "8"), ("6", "9"), ("9", "6"), ("1", "1"), ("0", "0")) if len(s) <= ln - 2] 
        return cnt if low != 0 else cnt + 1

# 0248 Hard 248 Strobogrammatic Number III

In [ ]:
# Time:  O(5^(n/2))
# Space: O(n)

class Solution(object):
    lookup = {'0':'0', '1':'1', '6':'9', '8':'8', '9':'6'}
    cache = {}

    # @param {string} low
    # @param {string} high
    # @return {integer}
    def strobogrammaticInRange(self, low, high):
        count = self.countStrobogrammaticUntil(high, False) - \
                self.countStrobogrammaticUntil(low, False) + \
                self.isStrobogrammatic(low)
        return count if count >= 0 else 0

    def countStrobogrammaticUntil(self, num,  can_start_with_0):
        if can_start_with_0 and num in self.cache:
            return self.cache[num]

        count = 0
        if len(num) == 1:
            for c in ['0', '1', '8']:
                if num[0] >= c:
                    count += 1
            self.cache[num] = count
            return count

        for key, val in self.lookup.iteritems():
            if can_start_with_0 or key != '0':
                if num[0] > key:
                    if len(num) == 2:  # num is like "21"
                        count += 1
                    else:  # num is like "201"
                        count += self.countStrobogrammaticUntil('9' * (len(num) - 2), True)
                elif num[0] == key:
                    if len(num) == 2:  # num is like 12".
                        if num[-1] >= val:
                            count += 1
                    else:
                        if num[-1] >= val:  # num is like "102".
                            count += self.countStrobogrammaticUntil(self.getMid(num), True)
                        elif (self.getMid(num) != '0' * (len(num) - 2)):  # num is like "110".
                            count += self.countStrobogrammaticUntil(self.getMid(num), True) - \
                                     self.isStrobogrammatic(self.getMid(num))

        if not can_start_with_0: # Sum up each length.
            for i in xrange(len(num) - 1, 0, -1):
                count += self.countStrobogrammaticByLength(i)
        else:
            self.cache[num] = count

        return count

    def getMid(self, num):
        return num[1:len(num) - 1]

    def countStrobogrammaticByLength(self, n):
        if n == 1:
            return 3
        elif n == 2:
            return 4
        elif n == 3:
            return 4 * 3
        else:
            return 5 * self.countStrobogrammaticByLength(n - 2)

    def isStrobogrammatic(self, num):
        n = len(num)
        for i in xrange((n+1) / 2):
            if num[n-1-i] not in self.lookup or \
               num[i] != self.lookup[num[n-1-i]]:
                return False
        return True

# 0265  Hard  265. Paint House II

In [ ]:
class Solution:
    def minCostII(self, costs):
        for i in range(1, len(costs)):
            for j in range(len(costs[0])): costs[i][j] += min(costs[i - 1][:j] + costs[i - 1][j + 1:])
        return costs and min(costs[-1]) or 0

# 0265 Hard 265 Paint House II

In [ ]:
# Time:  O(n * k)
# Space: O(k)

class Solution2(object):
    def minCostII(self, costs):
        """
        :type costs: List[List[int]]
        :rtype: int
        """
        return min(reduce(self.combine, costs)) if costs else 0

    def combine(self, tmp, house):
        smallest, k, i = min(tmp), len(tmp), tmp.index(min(tmp))
        tmp, tmp[i] = [smallest] * k, min(tmp[:i] + tmp[i+1:])
        return map(sum, zip(tmp, house))


class Solution2(object):
    def minCostII(self, costs):
        """
        :type costs: List[List[int]]
        :rtype: int
        """
        if not costs:
            return 0

        n = len(costs)
        k = len(costs[0])
        min_cost = [costs[0], [0] * k]
        for i in xrange(1, n):
            smallest, second_smallest = float("inf"), float("inf")
            for j in xrange(k):
                if min_cost[(i - 1) % 2][j] < smallest:
                    smallest, second_smallest = min_cost[(i - 1) % 2][j], smallest
                elif min_cost[(i - 1) % 2][j] < second_smallest:
                    second_smallest = min_cost[(i - 1) % 2][j]
            for j in xrange(k):
                min_j = smallest if min_cost[(i - 1) % 2][j] != smallest else second_smallest
                min_cost[i % 2][j] = costs[i][j] + min_j

        return min(min_cost[(n - 1) % 2])

# 0269  Hard  269. Alien Dictionary

In [ ]:
class Solution(object):
    def alienOrder(self, words):
        if len(words) == 1: return words[0]
        def dfs(i):
            visited[i] = 0
            for v in graph[i]:
                if visited[v] == 0 or (visited[v] == -1 and not dfs(v)): return False
            order.append(chr(97 + i))
            visited[i] = 1
            return True
        graph, visited, order = collections.defaultdict(set), [1] * 26, []
        for w1, w2 in zip(words, words[1:]):
            for c1, c2 in zip(w1, w2):
                if c1 != c2:
                    graph[ord(c1) - ord("a")].add(ord(c2) - ord("a"))
                    break
            for c in w1 + w2: visited[ord(c) - ord("a")] = -1
        for i in range(26):
            if visited[i] == -1 and not dfs(i): return ""
        return "".join(order)[::-1]

# 0269 Hard 269 Alien Dictionary

In [ ]:
"""
Topological Sort
"""
class Solution(object):
    def alienOrder(self, words):
        #return true if cycles are detected.
        def dfs(c):
            if c in path: return True
            if c in visited: return False
            path.add(c)
            for nei in adj[c]:
                if dfs(nei): return True
            res.append(c)
            path.remove(c)
            visited.add(c)
            return False
        
        #build adjacency list
        adj = {c: set() for word in words for c in word}
        for i in xrange(len(words)-1):
            w1, w2 = words[i], words[i+1]
            minLen = min(len(w1), len(w2))
            if w1[:minLen]==w2[:minLen] and len(w1)>len(w2): return ""
            
            for j in xrange(minLen):
                if w1[j]!=w2[j]:
                    adj[w1[j]].add(w2[j])
                    break
        
        #topological sort
        path = set() #path currently being reversed
        visited = set() #done processing
        res = []
        for c in adj:
            if dfs(c): return ""
            
        return "".join(reversed(res))

# 0272  Hard  272. Closest Binary Search Tree Value II

In [ ]:
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, x):
#         self.val = x
#         self.left = None
#         self.right = None

class Solution:
    def closestKValues(self, root, target, k):
        d = []
        def dfs(node):
            if node:
                heapq.heappush(d, (abs(node.val - target), node.val))
                dfs(node.left)
                dfs(node.right)
        dfs(root)
        return [node for val, node in heapq.nsmallest(k, d)]

# 0272 Hard 272 Closest Binary Search Tree Value II

In [ ]:
# Time:  O(h + k)
# Space: O(h)

class Solution(object):
    def closestKValues(self, root, target, k):
        """
        :type root: TreeNode
        :type target: float
        :type k: int
        :rtype: List[int]
        """
        # Helper to make a stack to the next node.
        def nextNode(stack, child1, child2):
            if stack:
                if child2(stack):
                    stack.append(child2(stack))
                    while child1(stack):
                        stack.append(child1(stack))
                else:
                    child = stack.pop()
                    while stack and child is child2(stack):
                        child = stack.pop()

        # The forward or backward iterator.
        backward = lambda stack: stack[-1].left
        forward = lambda stack: stack[-1].right

        # Build the stack to the closest node.
        stack = []
        while root:
            stack.append(root)
            root = root.left if target < root.val else root.right
        dist = lambda node: abs(node.val - target)
        forward_stack = stack[:stack.index(min(stack, key=dist))+1]

        # Get the stack to the next smaller node.
        backward_stack = list(forward_stack)
        nextNode(backward_stack, backward, forward)

        # Get the closest k values by advancing the iterators of the stacks.
        result = []
        for _ in xrange(k):
            if forward_stack and \
                (not backward_stack or dist(forward_stack[-1]) < dist(backward_stack[-1])):
                result.append(forward_stack[-1].val)
                nextNode(forward_stack, forward, backward)
            elif backward_stack and \
                (not forward_stack or dist(backward_stack[-1]) <= dist(forward_stack[-1])):
                result.append(backward_stack[-1].val)
                nextNode(backward_stack, backward, forward)
        return result


class Solution2(object):
    def closestKValues(self, root, target, k):
        """
        :type root: TreeNode
        :type target: float
        :type k: int
        :rtype: List[int]
        """
        # Helper class to make a stack to the next node.
        class BSTIterator:
            # @param root, a binary search tree's root node
            def __init__(self, stack, child1, child2):
                self.stack = list(stack)
                self.cur = self.stack.pop()
                self.child1 = child1
                self.child2 = child2

            # @return an integer, the next node
            def next(self):
                node = None
                if self.cur and self.child1(self.cur):
                    self.stack.append(self.cur)
                    node = self.child1(self.cur)
                    while self.child2(node):
                        self.stack.append(node)
                        node = self.child2(node)
                elif self.stack:
                    prev = self.cur
                    node = self.stack.pop()
                    while node:
                        if self.child2(node) is prev:
                            break
                        else:
                            prev = node
                            node = self.stack.pop() if self.stack else None
                self.cur = node
                return node

        # Build the stack to the closet node.
        stack = []
        while root:
            stack.append(root)
            root = root.left if target < root.val else root.right
        dist = lambda node: abs(node.val - target) if node else float("inf")
        stack = stack[:stack.index(min(stack, key=dist))+1]

        # The forward or backward iterator.
        backward = lambda node: node.left
        forward = lambda node: node.right
        smaller_it, larger_it = BSTIterator(stack, backward, forward), BSTIterator(stack, forward, backward)
        smaller_node, larger_node = smaller_it.next(), larger_it.next()

        # Get the closest k values by advancing the iterators of the stacks.
        result = [stack[-1].val]
        for _ in xrange(k - 1):
            if dist(smaller_node) < dist(larger_node):
                result.append(smaller_node.val)
                smaller_node = smaller_it.next()
            else:
                result.append(larger_node.val)
                larger_node = larger_it.next()
        return result

# 0273  Hard  273. Integer to English Words

In [ ]:
class Solution:
    def __init__(self):
        self.lessThan20 = ["","One","Two","Three","Four","Five","Six","Seven","Eight","Nine","Ten","Eleven","Twelve","Thirteen","Fourteen","Fifteen","Sixteen","Seventeen","Eighteen","Nineteen"]
        self.tens = ["","Ten","Twenty","Thirty","Forty","Fifty","Sixty","Seventy","Eighty","Ninety"]
        self.thousands = ["","Thousand","Million","Billion"]

    def numberToWords(self, num):
        if not num:
            return "Zero"
        res = ""
        for thousand in self.thousands:
            if num % 1000:
                res = self.helper(num%1000) + thousand + " " + res
            num //= 1000
        return res.strip()

    def helper(self, num):
        if not num:
            return ""
        elif num < 20:
            return self.lessThan20[num] + " "
        elif num < 100:
            return self.tens[num//10] + " " + self.helper(num%10)
        else:
            return self.lessThan20[num//100] + " Hundred " + self.helper(num%100)

# 0273 Hard 273. Integer to English Words

In [ ]:
class Solution(object):
    def numberToWords(self, num):
        # https://leetcode.com/problems/integer-to-english-words/discuss/70632/Recursive-Python
        to19 = 'One Two Three Four Five Six Seven Eight Nine Ten Eleven Twelve ' \
               'Thirteen Fourteen Fifteen Sixteen Seventeen Eighteen Nineteen'.split()
        tens = 'Twenty Thirty Forty Fifty Sixty Seventy Eighty Ninety'.split()
        def words(n):
            if n < 20:
                return to19[n - 1:n]
            if n < 100:
                return [tens[n / 10 - 2]] + words(n % 10)
            if n < 1000:
                return [to19[n / 100 - 1]] + ['Hundred'] + words(n % 100)
            for p, w in enumerate(('Thousand', 'Million', 'Billion'), 1):
                if n < 1000 ** (p + 1):
                    return words(n / 1000 ** p) + [w] + words(n % 1000 ** p)
        return ' '.join(words(num)) or 'Zero'

# 0273 Hard 273 Integer to English Words

In [ ]:
class Solution(object):
    def numberToWords(self, num):
        underTwenty = ['', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Eleven', 'Twelve', 'Thirteen', 'Fourteen', 'Fifteen', 'Sixteen', 'Seventeen', 'Eighteen', 'Nineteen', ]
        tens = ['', '', 'Twenty', 'Thirty', 'Forty', 'Fifty', 'Sixty', 'Seventy', 'Eighty', 'Ninety']
        
        def toWords(n):
            if n==0:
                return []
            elif n<20:
                return [underTwenty[n]]
            elif n<100:
                c, r = n/10, n%10
                return [tens[c]] + toWords(r)
            elif n<1000:
                c, r = n/100, n%100
                return toWords(c) + ['Hundred'] + toWords(r)
            else:
                if n>=1000000000:
                    c, r = n/1000000000, n%1000000000
                    return toWords(c) + ['Billion'] + toWords(r)
                elif n>=1000000:
                    c, r = n/1000000, n%1000000
                    return toWords(c) + ['Million'] + toWords(r)
                elif n>=1000:
                    c, r = n/1000, n%1000
                    return toWords(c) + ['Thousand'] + toWords(r)

        return ' '.join(toWords(num)) or 'Zero'

# 0282  Hard  282. Expression Add Operators

In [ ]:
class Solution:
    def addOperators(self, num, target):
        """
        :type num: str
        :type target: int
        :rtype: List[str]
        """
        # s, val, cur, coeff
        q = {("", 0, 0, 1)}
        for i, c in enumerate(num):
            new = set()
            for s, val, cur, coeff in q:
                if i:
                    new.add((s + "+" + c, val + int(c), int(c), 1))
                    new.add((s + "-" + c, val - int(c), int(c), -1))
                    new.add((s + "*" + c, val + cur * coeff * (int(c) - 1), int(c), cur * coeff))
                if s and s[-1] == "0" and cur == 0: continue
                pre = cur
                cur = cur * 10 + int(c)
                new.add((s + c, val + coeff * (cur - pre), cur, coeff))
            q = new
        return [s for s, val, cur, coeff in q if val == target]

# 0282 Hard 282 Expression Add Operators

In [ ]:
class Solution(object):
    def addOperators(self, num, target):
        def dfs(num, curr, i):
            if i==len(num):
                if eval(curr[1:])==target: ans.append(curr[1:])
                return
            
            if curr and curr[-1] in operators:
                dfs(num, curr+num[i], i+1)
            elif curr and curr[-1]=='0':
                dfs(num, curr+'+', i)
                dfs(num, curr+'-', i)
                dfs(num, curr+'*', i)
            else:
                dfs(num, curr+'+', i)
                dfs(num, curr+'-', i)
                dfs(num, curr+'*', i)
                dfs(num, curr+num[i], i+1)
        
        operators = set(['+', '-', '*'])
        ans = []
        dfs(num, '+', 0)
        return ans

# 0295  Hard  295. Find Median from Data Stream

In [ ]:
class MedianFinder:

    def __init__(self):
        self.l = []
        self.r = []
        self.m = []

    def addNum(self, num):
        if not self.m:
            self.m = [num]
        elif len(self.m) == 1:
            m = self.m[0]
            if num >= m:
                self.m = [m, heapq.heappushpop(self.r, num)]
            else:
                self.m = [-heapq.heappushpop(self.l, -num), m]
        else:
            m1, m2 = self.m
            if num >= m2:
                heapq.heappush(self.r, num)
                heapq.heappush(self.l, -m1)
                self.m = [m2]
            elif num <= m1:
                heapq.heappush(self.l, -num)
                heapq.heappush(self.r, m2)
                self.m = [m1]
            else:
                heapq.heappush(self.l, -m1)
                heapq.heappush(self.r, m2)
                self.m = [num]

    def findMedian(self):
        return sum(self.m) / len(self.m)

# 0295 Hard 295 Find Median from Data Stream

In [ ]:
"""
Time: O(N) for addNum() and O(1) for findMedian(). Though we can find the indext to insert in O(LogN) but insert still takes O(N)
Space: O(N).
"""
class MedianFinder(object):

    def __init__(self):
        self.a = []

    def addNum(self, num):
        bisect.insort_left(self.a, num)
        
    def findMedian(self):
        i = len(self.a)/2
        return (self.a[i-1]+self.a[i])/2.0 if len(self.a)%2==0 else self.a[i]


"""
Time: O(LogN) for addNum() and O(1) for findMedian().
Space: O(N).
"""
class MedianFinder(object):

    def __init__(self):
        self.smaller = [] #max heap
        self.larger = [] #min heap
        
    def addNum(self, num):
        if not self.smaller or num<-self.smaller[0]:
            heapq.heappush(self.smaller, -num)
        else:
            heapq.heappush(self.larger, num)
        
        if len(self.smaller)-len(self.larger)>1:
            heapq.heappush(self.larger, -heapq.heappop(self.smaller))
        elif len(self.larger)-len(self.smaller)>=1:
            heapq.heappush(self.smaller, -heapq.heappop(self.larger))
            
    def findMedian(self):
        return -self.smaller[0] if (len(self.smaller)-len(self.larger))%2!=0 else (-self.smaller[0]+self.larger[0])/2.0

# 0296  Hard  296. Best Meeting Point

In [ ]:
class Solution:
    def minTotalDistance(self, grid):
        m, n = len(grid), len(grid[0])
        x, y = sorted(i for i in range(m) for j in range(n) if grid[i][j]), sorted(j for i in range(m) for j in range(n) if grid[i][j])
        avg_x = len(x) % 2 and x[len(x) // 2] or (x[len(x) // 2 - 1] + x[len(x) // 2]) / 2
        avg_y = len(y) % 2 and y[len(y) // 2] or (y[len(y) // 2 - 1] + y[len(y) // 2]) / 2
        return int(sum(abs(avg_x - i) + abs(avg_y - j) for i, j in zip(x, y)))

# 0296 Hard 296. Best Meeting Point

In [ ]:
class Solution(object):
    # def minTotalDistance(self, grid):
    #     """
    #     :type grid: List[List[int]]
    #     :rtype: int
    #     """
    #     # sort
    #     rows = []
    #     cols = []
    #     for i in range(len(grid)):
    #         for j in range(len(grid[0])):
    #             if grid[i][j] == 1:
    #                 rows.append(i)
    #                 cols.append(j)
    #     row = rows[len(rows) / 2]
    #     cols.sort()
    #     col = cols[len(cols) / 2]
    #     return self.minDistance1D(rows, row) + self.minDistance1D(cols, col)

    # def minDistance1D(self, points, origin):
    #     distance = 0
    #     for point in points:
    #         distance += abs(point - origin)
    #     return distance

    def minDistance1D(self, points):
        # two points
        distance = 0
        i, j = 0, len(points) - 1
        while i < j:
            distance += points[j] - points[i]
            i += 1
            j -= 1
        return distance

    def minTotalDistance(self, grid):
        rows = self.collectRows(grid)
        cols = self.collectCols(grid)
        row = rows[len(rows) / 2]
        col = cols[len(cols) / 2]
        return self.minDistance1D(rows) + self.minDistance1D(cols)

    def collectRows(self, grid):
        rows = []
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                if grid[i][j] == 1:
                    rows.append(i)
        return rows

    def collectCols(self, grid):
        cols = []
        for j in range(len(grid[0])):
            for i in range(len(grid)):
                if grid[i][j] == 1:
                    cols.append(j)
        return cols

# 0296 Hard 296 Best Meeting Point

In [ ]:
# Time:  O(m * n)
# Space: O(m + n)

from random import randint


class Solution(object):
    def minTotalDistance(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        x = [i for i, row in enumerate(grid) for v in row if v == 1]
        y = [j for row in grid for j, v in enumerate(row) if v == 1]
        mid_x = self.findKthLargest(x, len(x) / 2 + 1)
        mid_y = self.findKthLargest(y, len(y) / 2 + 1)

        return sum([abs(mid_x-i) + abs(mid_y-j)
                   for i, row in enumerate(grid)
                   for j, v in enumerate(row) if v == 1])

    def findKthLargest(self, nums, k):
        left, right = 0, len(nums) - 1
        while left <= right:
            pivot_idx = randint(left, right)
            new_pivot_idx = self.PartitionAroundPivot(left, right,
                                                      pivot_idx, nums)
            if new_pivot_idx == k - 1:
                return nums[new_pivot_idx]
            elif new_pivot_idx > k - 1:
                right = new_pivot_idx - 1
            else:  # new_pivot_idx < k - 1.
                left = new_pivot_idx + 1

    def PartitionAroundPivot(self, left, right, pivot_idx, nums):
        pivot_value = nums[pivot_idx]
        new_pivot_idx = left
        nums[pivot_idx], nums[right] = nums[right], nums[pivot_idx]
        for i in xrange(left, right):
            if nums[i] > pivot_value:
                nums[i], nums[new_pivot_idx] = nums[new_pivot_idx], nums[i]
                new_pivot_idx += 1

        nums[right], nums[new_pivot_idx] = nums[new_pivot_idx], nums[right]
        return new_pivot_idx

# 0297  Hard  297. Serialize and Deserialize Binary Tree

In [ ]:
class Codec:

    def serialize(self, root):
        q, s = root and collections.deque([root]), ""
        while q:
            node = q.popleft()
            if node is None:
                s += "null#"
            else:
                s += str(node.val) + "#"
                q += [node.left, node.right]
        return s
        

    def deserialize(self, data):
        data = data and collections.deque(data.split("#"))
        q, root = data and collections.deque([TreeNode(int(data.popleft()))]), None
        while q:
            node = q.popleft()
            if not root:
                root = node
            l, r = data.popleft(), data.popleft()
            if l != "null":
                node.left = TreeNode(int(l))
                q.append(node.left)
            if r != "null":
                node.right = TreeNode(int(r))
                q.append(node.right)
        return root

# 0297 Hard 297 Serialize and Deserialize Binary Tree

In [ ]:
class Codec:

    def serialize(self, root):
        if not root: return ''
        q = collections.deque([root])
        s = ''
        while q:
            node = q.popleft()
            
            if node:
                s += str(node.val)
                q.append(node.left)
                q.append(node.right)
            else:
                s += '#'
            s += ','
        return s[:-1]
            
        
    def deserialize(self, data):
        if not data: return None
        
        data = data.split(',')
        root = TreeNode(data[0])
        q = collections.deque([root])
        i = 1
        
        while q and i<len(data):
            node = q.popleft()
            
            if data[i]!='#':
                node.left = TreeNode(data[i])
                q.append(node.left)
            i+=1
            
            if i>=len(data): continue
            if data[i]!='#':
                node.right = TreeNode(data[i])
                q.append(node.right)
            i += 1
            
        return root



#Preorder traversal
class Codec:
    def serialize(self, root):
        if not root: return '#,'
        return str(root.val) + ',' + self.serialize(root.left) + self.serialize(root.right)
    
    def deserialize(self, data):
        data = data.split(',')[::-1]
        return self.getNode(data)
    
    def getNode(self, data):
        if not data: return None
        val = data.pop()
        if val=='#': return None
        node = TreeNode(val)
        node.left = self.getNode(data)
        node.right = self.getNode(data)
        return node

# 0301  Hard  301. Remove Invalid Parentheses

In [ ]:
class Solution:
    def removeInvalidParentheses(self, s):
        l = r = 0
        for c in s:
            if c.isalpha(): continue
            if c == "(": l += 1 
            elif l: l -= 1
            else: r += 1
        q = {("", l, r, 0, 0)}
        for c in s:
            new = set()
            for st, l, r, lCur, rCur in q:
                if c == "(":
                    new.add((st + c, l, r, lCur + 1, rCur))
                    if l:
                        new.add((st, l - 1, r, lCur, rCur))
                elif c == ")":
                    if lCur:
                        new.add((st + c, l, r, lCur - 1, rCur))
                    else:
                        new.add((st + c, l, r, lCur, rCur + 1))
                    if r:
                        new.add((st, l, r - 1, lCur, rCur))
                else:
                    new.add((st + c, l, r, lCur, rCur))
            q = new
        return list({st for st, l, r, lCur, rCur in q if l == r == lCur == rCur == 0})

# 0301 Hard 301 Remove Invalid Parentheses

In [ ]:
class Solution(object):
    def removeInvalidParentheses(self, s):
        def dfs(s, curr, i, count):
            if count<0: return
            if len(curr)>self.maxLen: return
            
            if i>=len(s):
                if len(curr)==self.maxLen and count==0:
                    self.ans.append(curr)
                return
            
            if s[i]!='(' and s[i]!=')':
                dfs(s, curr+s[i], i+1, count)
            elif not curr or s[i]!=curr[-1]:
                dfs(s, curr+s[i], i+1, count + (1 if s[i]=='(' else -1))
                dfs(s, curr, i+1, count)
            elif s[i]==curr[-1]:
                dfs(s, curr+s[i], i+1, count + (1 if s[i]=='(' else -1))
                
        def getMaxLen(s):
            openCount = removeCount = 0
            for c in s:
                if c=='(':
                    openCount += 1
                elif c==')':
                    openCount -= 1
                
                if openCount<0:
                    removeCount += abs(openCount)
                    openCount = 0
            removeCount += openCount   
            return len(s)-removeCount
        
        self.ans = []
        self.maxLen = getMaxLen(s)
        
        dfs(s, "", 0, 0)
        return self.ans

# 0302  Hard  302. Smallest Rectangle Enclosing Black Pixels

In [ ]:
class Solution:
    def minArea(self, image, x, y):
        l, r, u, d, m, n = [y], [y], [x], [x], len(image), len(image[0])
        def dfs(i, j):
            if i < u[0]: u[0] = i
            elif i > d[0]: d[0] = i
            if j < l[0]: l[0] = j
            elif j > r[0]: r[0] = j
            for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
                if 0 <= x < m and 0 <= y < n and image[x][y] == "1":
                    image[x][y] = "0"
                    dfs(x, y)
        dfs(x, y)
        return (r[0] - l[0] + 1) * (d[0] - u[0] + 1)

# 0302 Hard 302 Smallest Rectangle Enclosing Black Pixels

In [ ]:
# Time:  O(nlogn)
# Space: O(1)

import bisect
import itertools


class Solution(object):
    def minArea(self, image, x, y):
        """
        :type image: List[List[str]]
        :type x: int
        :type y: int
        :rtype: int
        """
        def binarySearch(left, right, find, image, has_one):
            while left <= right:  # O(logn) times
                mid = left + (right - left) / 2
                if find(image, has_one, mid):  # Time: O(n)
                    right = mid - 1
                else:
                    left = mid + 1
            return left


        searchColumns = lambda image, has_one, mid: any([int(row[mid]) for row in image]) == has_one
        left = binarySearch(0, y - 1, searchColumns, image, True)
        right = binarySearch(y + 1, len(image[0]) - 1, searchColumns, image, False)

        searchRows = lambda image, has_one, mid: any(itertools.imap(int, image[mid])) == has_one
        top = binarySearch(0, x - 1, searchRows, image, True)
        bottom = binarySearch(x + 1, len(image) - 1, searchRows, image, False)

        return (right - left) * (bottom - top)

# 0305  Hard  305. Number of Islands II

In [ ]:
class Solution:
    def numIslands2(self, m, n, positions):
        def getParent(i):
            if i != parent[i]:
                parent[i] = getParent(parent[i])
            return parent[i]
        islands, res, parent, Id = set(), [], {}, 1
        for i, j in positions:
            parent[(i, j)] = parent[Id] = Id
            for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
                if (x, y) in parent:
                    p = getParent(parent[(x, y)])
                    islands.discard(p)
                    parent[p] = Id
            islands.add(Id)
            Id += 1
            res.append(len(islands))
        return res

# 0305 Hard 305. Number of Islands II

In [ ]:
class Solution(object):
    def numIslands2(self, m, n, positions):
        """
        :type m: int
        :type n: int
        :type positions: List[List[int]]
        :rtype: List[int]
        """
        # quick union find with weights
        ans = []
        islands = Union()
        for p in map(tuple, positions):
            islands.add(p)
            for dp in (0, 1), (0, -1), (1, 0), (-1, 0):
                q = (p[0] + dp[0], p[1] + dp[1])
                if q in islands.id:
                    islands.unite(p, q)
            ans += [islands.count]
        return ans

class Union(object):
    """
    quick union find with weights
    """
    def __init__(self):
        # use dic to reduce index operations
        self.id = {}
        self.sz = {}
        self.count = 0

    def add(self, p):
        self.id[p] = p
        self.sz[p] = 1
        self.count += 1

    def root(self, i):
        while i != self.id[i]:
            self.id[i] = self.id[self.id[i]]
            i = self.id[i]
        return i

    def unite(self, p, q):
        i, j = self.root(p), self.root(q)
        if i == j:
            return
        if self.sz[i] > self.sz[j]:
            i, j = j, i
        self.id[i] = j
        self.sz[j] += self.sz[i]
        self.count -= 1

# 0305 Hard 305 Number of Islands II

In [ ]:
"""
For each new land, the count will -= ((number of neighbors) - 1).
Because some for this new land introduced, some of the separate islands will be connected.
Note that, "number of neighbors", islands connected does not count. So for each neighbor, we use "find" to find its root.
"""
class Solution(object):
    def numIslands2(self, M, N, positions):
        def coorToNum(r, c):
            return N*r+c
        
        def find(n):
            p = parents[n]
            while p!=parents[p]:
                p = find(p)
            parents[n] = p
            return parents[n]
            
        def union(n1, n2):
            p1 = find(n1)
            p2 = find(n2)
            
            if p1==p2: return
            parents[p1] = p2
            
        count = 0
        lands = set()
        ans = []
        parents = [n for n in xrange(M*N)]
        
        for r, c in positions:
            n = coorToNum(r, c)
            
            if n in lands:
                ans.append(count)
                continue
            
            neis = set()
            if r+1<M and coorToNum(r+1, c) in lands: neis.add(find(coorToNum(r+1, c)))
            if 0<=r-1 and coorToNum(r-1, c) in lands: neis.add(find(coorToNum(r-1, c)))
            if c+1<N and coorToNum(r, c+1) in lands: neis.add(find(coorToNum(r, c+1)))
            if 0<=c-1 and coorToNum(r, c-1) in lands: neis.add(find(coorToNum(r, c-1)))
                
            count -= (len(neis)-1)
            lands.add(n)
            ans.append(count)
            
            for nei in neis:
                union(n, nei)
            
        return ans

# 0308  Hard  308. Range Sum Query 2D  Mutable

In [ ]:
class NumMatrix(object):
    def __init__(self, matrix):
        """
        initialize your data structure here.
        :type matrix: List[List[int]]
        """
        for row in matrix:
            for col in range(1, len(row)):
                row[col] += row[col-1]
        self.matrix = matrix
        

    def update(self, row, col, val):
        """
        update the element at matrix[row,col] to val.
        :type row: int
        :type col: int
        :type val: int
        :rtype: void
        """
        original = self.matrix[row][col]
        if col != 0:
            original -= self.matrix[row][col-1]
            
        diff = original - val
        
        for y in range(col, len(self.matrix[0])):
            self.matrix[row][y] -= diff

    def sumRegion(self, row1, col1, row2, col2):
        """
        sum of elements matrix[(row1,col1)..(row2,col2)], inclusive.
        :type row1: int
        :type col1: int
        :type row2: int
        :type col2: int
        :rtype: int
        """
        sum = 0
        for x in range(row1, row2+1):
            sum += self.matrix[x][col2]
            if col1 != 0:
                sum -= self.matrix[x][col1-1]
        return sum

# 0312  Hard  312. Burst Balloons

In [ ]:
class Solution:
    def maxCoins(self, nums):
        memo, nums = {}, [1] + [num for num in nums if num] + [1]
        def dfs(l, r):
            if r - l == 1: return 0
            if (l, r) not in memo: memo[(l, r)] = max(nums[l] * nums[i] * nums[r] + dfs(l, i) + dfs(i, r) for i in range(l + 1, r))
            return memo[(l, r)]
        return dfs(0, len(nums) - 1)

# 0312 Hard 312 Burst Balloons

In [ ]:
"""
dp[i][j] := max coins gains from nums[i+1:j]
Assume k is the last balloon that get burst within nums[i:j+1], try different k and get the max dp[i][j].

max coins gains from nums[i+1:j] will be the sum of
1. max coins gains from nums[i:k]: `dp[i][k-1] if k-1>=0 else 0`
2. coins gains from bursring k: `(nums[i-1] if 0<=i-1 else 1) * nums[k] * (nums[j+1] if j+1<N else 1)`
3. max coins gains from nums[k+1:j+1]: `dp[k+1][j] if k+1<N else 0`
"""
class Solution(object):
    def maxCoins(self, nums):
        N = len(nums)
        
        dp = [[0 for _ in xrange(N)] for _ in xrange(N)]
        for i in xrange(N): dp[i][i] = (nums[i-1] if 0<=i-1 else 1) * nums[i] * (nums[i+1] if i+1<N else 1)
            
        for l in xrange(2, N+1):
            for i in xrange(N):
                j = i+l-1
                if j>=N: continue
                for k in xrange(i, j+1):
                    dp[i][j] = max(dp[i][j],
                        (dp[i][k-1] if k-1>=0 else 0) +
                        (nums[i-1] if 0<=i-1 else 1) * nums[k] * (nums[j+1] if j+1<N else 1) +
                        (dp[k+1][j] if k+1<N else 0)
                    )
        
        return dp[0][N-1]

# 0315  Hard  315. Count of Smaller Numbers After Self

In [ ]:
class Solution:
    def countSmaller(self, nums):
        r = []
        for i in range(len(nums) - 1, -1, -1):
            bisect.insort(r, nums[i])
            nums[i] = bisect.bisect_left(r, nums[i])
        return nums

# 0315 Hard 315 Count of Smaller Numbers After Self

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class Solution(object):
    def countSmaller(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        def countAndMergeSort(num_idxs, start, end, counts):
            if end - start <= 0:  # The size of range [start, end] less than 2 is always with count 0.
                return

            mid = start + (end - start) // 2
            countAndMergeSort(num_idxs, start, mid, counts)
            countAndMergeSort(num_idxs, mid + 1, end, counts)
            r = mid + 1
            tmp = []
            for i in xrange(start, mid + 1):
                # Merge the two sorted arrays into tmp.
                while r <= end and num_idxs[r][0] < num_idxs[i][0]:
                    tmp.append(num_idxs[r])
                    r += 1
                tmp.append(num_idxs[i])
                counts[num_idxs[i][1]] += r - (mid + 1)

            # Copy tmp back to num_idxs
            num_idxs[start:start+len(tmp)] = tmp

        num_idxs = []
        counts = [0] * len(nums)
        for i, num in enumerate(nums):
            num_idxs.append((num, i))
        countAndMergeSort(num_idxs, 0, len(num_idxs) - 1, counts)
        return counts


# Time:  O(nlogn)
# Space: O(n)
# BIT solution.
class Solution2(object):
    def countSmaller(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        class BIT(object):  # 0-indexed.
            def __init__(self, n):
                self.__bit = [0]*(n+1)  # Extra one for dummy node.

            def add(self, i, val):
                i += 1  # Extra one for dummy node.
                while i < len(self.__bit):
                    self.__bit[i] += val
                    i += (i & -i)

            def query(self, i):
                i += 1  # Extra one for dummy node.
                ret = 0
                while i > 0:
                    ret += self.__bit[i]
                    i -= (i & -i)
                return ret

        # Get the place (position in the ascending order) of each number.
        sorted_nums = sorted(zip(nums, range(len(nums))))
        lookup = {i:new_i for new_i, (_, i) in enumerate(sorted_nums)}

        # Count the smaller elements after the number.
        result, bit = [0]*len(nums), BIT(len(nums))
        for i in reversed(xrange(len(nums))):
            result[i] = bit.query(lookup[i]-1)
            bit.add(lookup[i], 1)
        return result


# Time:  O(nlogn) ~ O(n^2)
# Space: O(n)
# BST solution.
class Solution3(object):
    def countSmaller(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        res = [0] * len(nums)
        bst = self.BST()
        # Insert into BST and get left count.
        for i in reversed(xrange(len(nums))):
            bst.insertNode(nums[i])
            res[i] = bst.query(nums[i])

        return res

    class BST(object):
        class BSTreeNode(object):
            def __init__(self, val):
                self.val = val
                self.count = 0
                self.left = self.right = None

        def __init__(self):
            self.root = None

        # Insert node into BST.
        def insertNode(self, val):
            node = self.BSTreeNode(val)
            if not self.root:
                self.root = node
                return
            curr = self.root
            while curr:
                # Insert left if smaller.
                if node.val < curr.val:
                    curr.count += 1  # Increase the number of left children.
                    if curr.left:
                        curr = curr.left
                    else:
                        curr.left = node
                        break
                else:  # Insert right if larger or equal.
                    if curr.right:
                        curr = curr.right
                    else:
                        curr.right = node
                        break

        # Query the smaller count of the value.
        def query(self, val):
            count = 0
            curr = self.root
            while curr:
                # Insert left.
                if val < curr.val:
                    curr = curr.left
                elif val > curr.val:
                    count += 1 + curr.count  # Count the number of the smaller nodes.
                    curr = curr.right
                else:  # Equal.
                    return count + curr.count
            return 0

# 0317  Hard  317. Shortest Distance from All Buildings

In [ ]:
class Solution:
    def shortestDistance(self, grid: List[List[int]]) -> int:
        m, n, d = len(grid), len(grid[0]), 1
        piled = collections.defaultdict(set)
        dist = collections.defaultdict(int)
        bfs = [(i, j, i, j) for i in range(m) for j in range(n) if grid[i][j] == 1]
        total, res = len(bfs), []
        while bfs:
            new = []
            for x, y, i, j in bfs:
                for ii, jj in ((i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)):
                    if 0 <= ii < m and 0 <= jj < n and not grid[ii][jj] and (x, y) not in piled[(ii, jj)]:
                        piled[(ii, jj)].add((x, y))
                        dist[(ii, jj)] += d
                        if len(piled[(ii, jj)]) == total:
                            res.append(dist[(ii, jj)])
                        new.append((x, y, ii, jj))
            bfs = new
            d += 1
        return min(res) if res else -1

# 0317 Hard 317 Shortest Distance from All Buildings

In [ ]:
class Solution(object):
    def shortestDistance(self, grid):
        def bfs(i0, j0):
            q = collections.deque([(i0, j0, 0)])
            visited = set()
            while q:
                i, j, dis = q.popleft()
                if (i, j) in visited: continue
                visited.add((i, j))
                
                reach[i][j] += 1
                distances[i][j] += dis
                
                for iNext, jNext in ((i+1, j), (i-1, j), (i, j+1), (i, j-1)):
                    if not (0<=iNext<len(grid) and 0<=jNext<len(grid[0])): continue
                    if grid[iNext][jNext]==2 or grid[iNext][jNext]==1: continue
                    if (iNext, jNext) in visited: continue
                    q.append((iNext, jNext, dis+1))
        
        M = len(grid)
        N = len(grid[0])
        ans = float('inf')
        reach = [[0]*N for _ in xrange(M)] #number of buildings can reach (i, j)
        distances = [[0]*N for _ in xrange(M)] #aggregate distance btwn 0 and buildings.
        buildingCount = 0
        
        for i in xrange(M):
            for j in xrange(N):
                if grid[i][j]==1:
                    buildingCount += 1
                    bfs(i, j)
        
        
        for i in xrange(M):
            for j in xrange(N):
                if grid[i][j]==0 and reach[i][j]==buildingCount:
                    ans = min(ans, distances[i][j])
        
        return ans if ans!=float('inf') else -1

# 0321  Hard  321. Create Maximum Number

In [ ]:
class Solution:
    def maxNumber(self, nums1, nums2, k):
        def merge(arr1, arr2):
            res, i, j = [], 0, 0
            while i < len(arr1) and j < len(arr2):
                if arr1[i:] >= arr2[j:]:
                    res.append(arr1[i])
                    i += 1
                else: 
                    res.append(arr2[j])
                    j += 1
            if i < len(arr1): res += arr1[i:]
            elif j < len(arr2): res += arr2[j:]
            return res     
        
        def makeArr(arr, l):
            i, res = 0, []
            for r in range(l - 1, -1, -1):
                num, i = max(arr[i:-r] or arr[i:])
                i = -i + 1
                res.append(num)
            return res
        
        nums1, nums2, choices = [(num, -i) for i, num in enumerate(nums1)], [(num, -i) for i, num in enumerate(nums2)], []
        for m in range(k + 1):
            if m > len(nums1) or k - m > len(nums2): continue
            arr1, arr2 = makeArr(nums1, m), makeArr(nums2, k - m)  
            choices.append(merge(arr1, arr2))
        return max(choices)
            

# 0321 Hard 321 Create Maximum Number

In [ ]:
# Time:  O(k * (m + n + k)) ~ O(k * (m + n + k^2))
# Space: O(m + n + k^2)

class Solution(object):
    def maxNumber(self, nums1, nums2, k):
        """
        :type nums1: List[int]
        :type nums2: List[int]
        :type k: int
        :rtype: List[int]
        """
        def get_max_digits(nums, start, end, max_digits):
            max_digits[end] = max_digit(nums, end)
            for i in reversed(xrange(start, end)):
                max_digits[i] = delete_digit(max_digits[i + 1])

        def max_digit(nums, k):
            drop = len(nums) - k
            res = []
            for num in nums:
                while drop and res and res[-1] < num:
                    res.pop()
                    drop -= 1
                res.append(num)
            return res[:k]

        def delete_digit(nums):
            res = list(nums)
            for i in xrange(len(res)):
                if i == len(res) - 1 or res[i] < res[i + 1]:
                    res = res[:i] + res[i+1:]
                    break
            return res

        def merge(a, b):
            return [max(a, b).pop(0) for _ in xrange(len(a)+len(b))]

        m, n = len(nums1), len(nums2)

        max_digits1, max_digits2 = [[] for _ in xrange(k + 1)], [[] for _ in xrange(k + 1)]
        get_max_digits(nums1, max(0, k - n), min(k, m), max_digits1)
        get_max_digits(nums2, max(0, k - m), min(k, n), max_digits2)

        return max(merge(max_digits1[i], max_digits2[k-i]) \
                   for i in xrange(max(0, k - n), min(k, m) + 1))

# 0327  Hard  327. Count of Range Sum

In [ ]:
class Solution:
    def countRangeSum(self, nums, lower, upper):
        sums, sm, res = [0], 0, 0
        for num in nums:
            sm += num
            res += bisect.bisect_right(sums, sm - lower) - bisect.bisect_left(sums, sm - upper)
            bisect.insort(sums, sm)
        return res

# 0327 Hard 327 Count of Range Sum

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class Solution(object):
    def countRangeSum(self, nums, lower, upper):
        """
        :type nums: List[int]
        :type lower: int
        :type upper: int
        :rtype: int
        """
        def countAndMergeSort(sums, start, end, lower, upper):
            if end - start <= 1:  # The size of range [start, end) less than 2 is always with count 0.
                return 0
            mid = start + (end - start) / 2
            count = countAndMergeSort(sums, start, mid, lower, upper) + \
                    countAndMergeSort(sums, mid, end, lower, upper)
            j, k, r = mid, mid, mid
            tmp = []
            for i in xrange(start, mid):
                # Count the number of range sums that lie in [lower, upper].
                while k < end and sums[k] - sums[i] < lower:
                    k += 1
                while j < end and sums[j] - sums[i] <= upper:
                    j += 1
                count += j - k

                # Merge the two sorted arrays into tmp.
                while r < end and sums[r] < sums[i]:
                    tmp.append(sums[r])
                    r += 1
                tmp.append(sums[i])
            # Copy tmp back to sums.
            sums[start:start+len(tmp)] = tmp
            return count

        sums = [0] * (len(nums) + 1)
        for i in xrange(len(nums)):
            sums[i + 1] = sums[i] + nums[i]
        return countAndMergeSort(sums, 0, len(sums), lower, upper)


# Divide and Conquer solution.
class Solution2(object):
    def countRangeSum(self, nums, lower, upper):
        """
        :type nums: List[int]
        :type lower: int
        :type upper: int
        :rtype: int
        """
        def countAndMergeSort(sums, start, end, lower, upper):
            if end - start <= 0:  # The size of range [start, end] less than 2 is always with count 0.
                return 0

            mid = start + (end - start) / 2
            count = countAndMergeSort(sums, start, mid, lower, upper) + \
                    countAndMergeSort(sums, mid + 1, end, lower, upper)
            j, k, r = mid + 1, mid + 1, mid + 1
            tmp = []
            for i in xrange(start, mid + 1):
                # Count the number of range sums that lie in [lower, upper].
                while k <= end and sums[k] - sums[i] < lower:
                    k += 1
                while j <= end and sums[j] - sums[i] <= upper:
                    j += 1
                count += j - k

                # Merge the two sorted arrays into tmp.
                while r <= end and sums[r] < sums[i]:
                    tmp.append(sums[r])
                    r += 1
                tmp.append(sums[i])

            # Copy tmp back to sums
            sums[start:start+len(tmp)] = tmp
            return count

        sums = [0] * (len(nums) + 1)
        for i in xrange(len(nums)):
            sums[i + 1] = sums[i] + nums[i]
        return countAndMergeSort(sums, 0, len(sums) - 1, lower, upper)

# 0329  Hard  329. Longest Increasing Path in a Matrix

In [ ]:
class Solution:
    def longestIncreasingPath(self, matrix):
        def dfs(i, j):
            if not dp[i][j]:
                dp[i][j] = 1+max((dfs(x,y) for x,y in ((i-1,j),(i+1,j),(i,j-1),(i,j+1)) if 0 <=x<m and 0<=y<n and matrix[x][y] > matrix[i][j]),default=0)
            return dp[i][j]
        m, n, = len(matrix), len(matrix and matrix[0])
        dp = [[0] * n for _ in range(m)]
        return max((dfs(i,j) for i in range(m) for j in range(n)),default=0)

# 0329 Hard 329 Longest Increasing Path in a Matrix

In [ ]:
class Solution(object):
    def longestIncreasingPath(self, matrix):
        def getLongest(i, j):
            if (i, j) in longest: return longest[(i, j)]
            l = 1

            #call getLongest to the neighbors that are larger than itself.
            if i+1<M and matrix[i][j]<matrix[i+1][j]:
                l = max(l, 1+getLongest(i+1, j))
            if i-1>=0 and matrix[i][j]<matrix[i-1][j]:
                l = max(l, 1+getLongest(i-1, j))
            if j+1<N and matrix[i][j]<matrix[i][j+1]:
                l = max(l, 1+getLongest(i, j+1))
            if j-1>=0 and matrix[i][j]<matrix[i][j-1]:
                l = max(l, 1+getLongest(i, j-1))
            
            longest[(i, j)] = l
            self.ans = max(self.ans, l)
            return l

        M = len(matrix)
        N = len(matrix[0])
        
        longest = {}
        self.ans = float('-inf')
        
        for i in xrange(M):
            for j in xrange(N):
                getLongest(i, j)
                
        return self.ans

# 0330  Hard  330. Patching Array

In [ ]:
class Solution:
    def minPatches(self, nums, n):
        miss, added, i = 1, 0, 0
        while miss <= n:
            if i < len(nums) and nums[i] <= miss:
                miss += nums[i]
                i += 1
            else:
                miss *= 2
                added += 1
        return added

# 0330 Hard 330 Patching Array

In [ ]:
# Time:  O(s + logn), s is the number of elements in the array
# Space: O(1)

class Solution(object):
    def minPatches(self, nums, n):
        """
        :type nums: List[int]
        :type n: int
        :rtype: int
        """
        patch, miss, i = 0, 1, 0
        while miss <= n:
            if i < len(nums) and nums[i] <= miss:
                miss += nums[i]
                i += 1
            else:
                miss += miss
                patch += 1

        return patch

# 0332  Hard  332. Reconstruct Itinerary

In [ ]:
class Solution:
    def findItinerary(self, tickets):
        graph, stack, reached = collections.defaultdict(list), ["JFK"], []
        for a, b in tickets: heapq.heappush(graph[a], b)  
        while stack:
            if graph[stack[-1]]: stack.append(heapq.heappop(graph[stack[-1]]))
            else: reached.append(stack.pop())
        return reached[::-1]

# 0332 Hard 332 Reconstruct Itinerary

In [ ]:
#build graph with departure city as key and (destination, ticket_id) as value
#G = build_graph()
#itinerary = ['JFK']
#ticket_used = set()
#dfs()
    #if len(itinerary)==len(tickets)+1: return True
    #here = itinerary[-1]
    #if here not in G: return False
    #candidates = nei, ticket_id for nei, ticket_id in G[here] if ticket_id not in ticket_used
    #for nei, ticket_id in candidates:
        #ticket_used.add(ticket_id)
        #itinerary.append(nei)
        #if dfs(): return True
        #ticket_used.remove(ticket_id)
        #itinerary.pop()
    #return False

import collections

class Solution(object):
    def findItinerary(self, tickets):
        def dfs():
            if len(itinerary)==len(tickets)+1: return True
            here = itinerary[-1]
            if here not in G: return False

            candidates = [(nei, ticket_id) for nei, ticket_id in G[here] if ticket_id not in ticket_used]
            
            for nei, ticket_id in candidates:
                ticket_used.add(ticket_id)
                itinerary.append(nei)
                if dfs(): return True
                ticket_used.remove(ticket_id)
                itinerary.pop()
            return False

        G = collections.defaultdict(list)
        itinerary = ['JFK']
        ticket_used = set()

        #build graph
        ticket_id_counter = 0
        for n1, n2 in tickets:
            G[n1].append((n2, ticket_id_counter))
            ticket_id_counter += 1
        for k in G: G[k].sort(key=lambda x:x[0])
        
        return dfs()

# 0335  Hard  335. Self Crossing

In [ ]:
class Solution:
    def isSelfCrossing(self, x: List[int]) -> bool:
        b = c = d = e = 0
        for a in x:
            if d >= b > 0 and (a >= c or a >= c-e >= 0 and f >= d-b):
                return True
            b, c, d, e, f = a, b, c, d, e
        return False

# 0335 Hard 335 Self Crossing

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def isSelfCrossing(self, x):
        """
        :type x: List[int]
        :rtype: bool
        """
        if len(x) >= 5 and x[3] == x[1] and x[4] + x[0] >= x[2]:
            # Crossing in a loop:
            #     2
            # 3 ââââââ
            #   ââââ>â1
            #   4  0  (overlapped)
            return True

        for i in xrange(3, len(x)):
            if x[i] >= x[i - 2] and x[i - 3] >= x[i - 1]:
                # Case 1:
                #    i-2
                # i-1âââ
                #    âââ¼â>i
                #     i-3
                return True
            elif i >= 5 and x[i - 4] <= x[i - 2] and x[i] + x[i - 4] >= x[i - 2] and \
                            x[i - 1] <= x[i - 3] and x[i - 5] + x[i - 1] >= x[i - 3]:
                # Case 2:
                #    i-4
                #    ââââ
                #    âi<â¼ââ
                # i-3â i-5âi-1
                #    ââââââ
                #      i-2
                return True
        return False

# 0336  Hard  336. Palindrome Pairs

In [ ]:
class Solution:
    def palindromePairs(self, words):
        index, res = {w:i for i, w in enumerate(words)}, []
        for i, w in enumerate(words):
            for j in range(len(w) + 1):
                pre, suf = w[:j], w[j:]
                if pre == pre[::-1]:
                    suf = suf[::-1]
                    if suf != w and suf in index:
                        res.append([index[suf], i])
                if j != len(w) and suf == suf[::-1]:
                    pre = pre[::-1]
                    if pre != w and pre in index:
                        res.append([i, index[pre]])
        return res

# 0336 Hard 336. Palindrome Pairs

In [ ]:
class Solution(object):
    def palindromePairs(self, words):
        """
        :type words: List[str]
        :rtype: List[List[int]]
        """
        # https://leetcode.com/problems/palindrome-pairs/discuss/79219/Python-solution~
        # reverse word and create a word to index map
        word2index, res = dict([(w[::-1], i) for i, w in enumerate(words)]), []
        for i, word in enumerate(words):
            for j in xrange(len(word) + 1):
                # Use prefix and postfix
                # rather than going through all posible combinations
                prefix, postfix = word[:j], word[j:]
                # prefix + postfix + reverse(prfix)
                if prefix in word2index and i != word2index[prefix] and postfix == postfix[::-1]:
                    res.append([i, word2index[prefix]])
                # reverse(postfix) + prefix + postfix
                if j > 0 and postfix in word2index and i != word2index[postfix] and prefix == prefix[::-1]:
                    res.append([word2index[postfix], i])
        return res

# 0336 Hard 336 Palindrome Pairs

In [ ]:
"""
Time: O(N x L^2), assume the average length of word is L. Note that, eversing the string takes O(L).
Space: O(L), beside from ans, need `left` and `right` constantly to store the word.
"""
class Solution(object):
    def palindromePairs(self, words):
        ans = set()
        index = {word:i for i, word in enumerate(words)}
        
        for i, word in enumerate(words):
            for j in xrange(len(word)+1):
                left = word[:j]
                right = word[j:]
                
                # check if any other word that concat to the left will make palindrome: "OTHER_WORD+`left`+`right`"
                # The above will be palindrome only if
                # 1. `left` is palindrome (left==left[::-1])
                # 2. Exsit an "OTHER_WORD" in word in words that equals to the reverse of `right` (right[::-1] in index and index[right[::-1]]!=i).
                if left==left[::-1]:
                    if right[::-1] in index and index[right[::-1]]!=i:
                        ans.add((index[right[::-1]], i))
                
                # check if any other word that concat to the right will make palindrome: "`left`+`right`+OTHER_WORD"
                # The above will be palindrome only if
                # 1. `rihgt` is palindrome (right==right[::-1])
                # 2. Exsit an "OTHER_WORD" in words that equals to the reverse of `left` (left[::-1] in index and index[left[::-1]]!=i).
                if right==right[::-1]:
                    if left[::-1] in index and index[left[::-1]]!=i:
                        ans.add((i, index[left[::-1]]))
                        
        return ans


"""
Time: O(N^2 x L), will cause TLE.
Space: O(L), can reduce to O(1) by improving isPalindrome()
"""
class Solution(object):
    def palindromePairs(self, words):
        N = len(words)
        ans = []
        
        for i in xrange(N):
            for j in xrange(N):
                if i==j: continue
                if self.isPalindrome(words[i]+words[j]):
                    ans.append([i, j])
        return ans
    
    def isPalindrome(self, word):
        l = 0
        r = len(word)-1
        
        while l<=r:
            if word[l]==word[r]:
                l += 1
                r -= 1
            else:
                return False
        return True
        
            

# 0352  Hard  352. Data Stream as Disjoint Intervals

In [ ]:
class SummaryRanges:

    def __init__(self):
        self.starts, self.ends, self.used = [-float("inf"), float("inf")], [-float("inf"), float("inf")], set()
        
    def addNum(self, val):
        if val not in self.used:
            self.used.add(val)
            i = bisect.bisect_left(self.starts, val) - 1
            if self.ends[i] + 1 == val and val + 1 == self.starts[i + 1]: # if val is the gap btw 2 intervals
                del self.starts[i + 1]
                del self.ends[i]
            elif self.ends[i] + 1 == val: #if val is adjacent to current end
                self.ends[i] += 1
            elif self.starts[i + 1] == val + 1: # if val is adjacent to next start
                self.starts[i + 1] -= 1
            elif val > self.ends[i]: # if val is independent of those 2 intervals
                self.starts.insert(i + 1, val) 
                self.ends.insert(i + 1, val)
            
    def getIntervals(self):
        return [[s, e] for s, e in zip(self.starts[1:-1], self.ends[1:-1])] #exclude infinity

# 0352 Hard 352 Data Stream as Disjoint Intervals

In [ ]:
class SummaryRanges(object):

    def __init__(self):
        self.nums = []
        

    def addNum(self, val):
        bisect.insort_left(self.nums, val)
        

    def getIntervals(self):
        ans = []
        
        for num in self.nums:
            if not ans:
                ans.append([num, num])
            elif ans[len(ans)-1][1]+1==num:
                ans[len(ans)-1][1] = num
            elif ans[len(ans)-1][1]<num:
                ans.append([num, num])
        return ans


"""
addNum() Time: O(N), Though finding the insertion point only takes O(LogN), insert to array still takes O(N).
getIntervals() Time: O(1)
Space: O(1)
"""
class SummaryRanges(object):

    def __init__(self):
        self.ans = []
    
    def addNum(self, val):
        if not self.ans:
            self.ans.append([val, val])
            return
            
        l = 0
        r = len(self.ans)-1
        
        #find insertion point
        while l<r:
            m = (l+r)/2
            if self.checkAndAdd(l, val): return
            if self.checkAndAdd(r, val): return
            if self.checkAndAdd(m, val): return
            
            if val<self.ans[m][0]:
                r = m
            else:
                l = m+1
        
        #insert val
        if self.checkAndAdd(l, val):
            return
        elif val<self.ans[l][0]:
            self.ans.insert(l, [val, val])
        else:
            self.ans.insert(l+1, [val, val])
        
    def getIntervals(self):
        return self.ans
    
    #add if val can add to self.ans[i]
    def checkAndAdd(self, i, val):
        if val==self.ans[i][0]-1:
            self.ans[i][0] = val

            #merge i-1 and i if needed
            if i>0 and self.ans[i-1][1]+1==self.ans[i][0]:
                self.ans[i][0] = self.ans[i-1][0]
                self.ans.pop(i-1)

            return True

        elif val==self.ans[i][1]+1:
            self.ans[i][1] = val

            #merge i and i+1 if needed
            if i+1<len(self.ans) and self.ans[i][1]+1==self.ans[i+1][0]:
                self.ans[i][1] = self.ans[i+1][1]
                self.ans.pop(i+1)

            return True

        elif self.ans[i][0]<=val and val<=self.ans[i][1]:
            return True

        return False
        

# 0354  Hard  354. Russian Doll Envelopes

In [ ]:
class Solution:
    def maxEnvelopes(self, envelopes):
        tails = []
        for w, h in sorted(envelopes, key = lambda x: (x[0], -x[1])):
            i = bisect.bisect_left(tails, h)
            if i == len(tails): tails.append(h)
            else: tails[i] = h
        return len(tails)

# 0354 Hard 354 Russian Doll Envelopes

In [ ]:
"""
dp[i] := number of envelopes that envelopes[i] can contains the most.
Very similar to LIS problem (Leetcode 300)
This method will get TLE

Time: O(N^2)
Space: O(N)
"""
class Solution(object):
    def maxEnvelopes(self, envelopes):
        if not envelopes: return 0
        
        N = len(envelopes)
        dp = [1]*N
        envelopes = sorted(envelopes)
        ans = 1
        
        for i in xrange(N):
            for j in xrange(i):
                if envelopes[i][0]>envelopes[j][0] and envelopes[i][1]>envelopes[j][1]:
                    dp[i] = max(dp[i], dp[j]+1)
                    ans = max(ans, dp[i])
        
        return ans

"""
dp[i] := the smallest ending number of a sequence that has length i+1
First, sort the envelopes by width. (envelope[0])
Second, return the LIS of heights ([envelope[1] for envelope in envelopes]).
This automatically gets the number of "russian dolls".

However, we might choose [3,4], [3,7] as a valid solution. This is not correct because w1==w2.
So we do a quick fix by sorting the envelopes with w and h reversed.
So that getLIS() will not choose [3,4], [3,7]

Time: O(NLogN)
Space: O(N)
"""
import bisect
class Solution(object):
    def maxEnvelopes(self, envelopes):
        if not envelopes: return 0
        
        N = len(envelopes)
        dp = [1]*N
        envelopes = envelopes.sort(key=lambda x:(x[0], -x[1]))

        return self.getLIS([envelope[1] for envelope in envelopes])
    
    def getLIS(self, A):
        dp = []

        for n in A:
            i = bisect.bisect_left(dp, n)

            if i==len(dp):
                dp.append(n)
            else:
                dp[i] = n
        
        return len(dp)

# 0358  Hard  358. Rearrange String k Distance Apart

In [ ]:
class Solution:
    def rearrangeString(self, s, k):
        q, last, res, wq = [(-v, k) for k, v in collections.Counter(s).items()], {}, "", []
        heapq.heapify(q)
        for i in range(len(s)):
            if wq and (wq[0][1] not in last or last[wq[0][1]] + k <= i): cnt, char = heapq.heappop(wq)
            else:
                while q and not (q[0][1] not in last or last[q[0][1]] + k <= i): heapq.heappush(wq, heapq.heappop(q))
                if not q: return ""
                cnt, char = heapq.heappop(q)
            res, cnt, last[char] = res + char, cnt + 1, i
            if cnt: heapq.heappush(q, (cnt, char))
        return res

# 0358 Hard 358 Rearrange String k Distance Apart

In [ ]:
"""
Time: O(N). N is the number of char in string s. See below explanation.
Space: O(1)

The main idea is to add k different char to the ans every time.
The char with higher count will be used first.
So there will be more different kind of char left and will not be too close to each other.

To do this, we need a counter to track count of each char. [0]
And a heap so we can get the char with highest count fast (LogH). [0]
For each time, add k different char to the ans. [1]
Char with count>0 will be put back to h. [2]

If heap is drained before we add k element to the ans. It is not possible to form ans. Return emptry string. [3]

Time complexity:
Contruct counter takes O(N).
Contruct the heap (heapify) takes O(H), H is the number of element in the heap.
In the while loop, each char in s will be pop once. O(NLogH).
Since the problem said that the string will only have lower case letters. So H will at most be 26.
So O(N+H+NlogH) ~= O(N+26+NLog26) ~= O(N)
"""
class Solution(object):
    def rearrangeString(self, s, k):
        if k==0: return s
        ans = ''
        
		#[0]
        counter = collections.Counter(s)
        h = [(-counter[c], c) for c in counter]
        heapq.heapify(h)
        
        while h:
            l = []
            for i in xrange(k):
                _, c = heapq.heappop(h)
                ans += c
                counter[c] -= 1
                if counter[c]!=0: l.append((-counter[c], c)) #[2]
                
                if len(ans)==len(s): return ans
                if not h and i!=k-1: return '' #[3]
            
            for e in l: heapq.heappush(h, e)
        
        return ans #this line should never be executed

# 0363  Hard  363. Max Sum of Rectangle No Larger Than K

In [ ]:
class Solution:
    def maxSumSubmatrix(self, matrix, k, mxTotal = -float("inf")):
        for l in range(len(matrix[0])):
            dp = [0] * len(matrix)
            for r in range(l, len(matrix[0])):
                for i in range(len(matrix)):
                    dp[i] += matrix[i][r]
                sums, cur, mx = [float("inf")], 0, -float("inf")
                for sm in dp:
                    bisect.insort(sums, cur)
                    cur += sm
                    mx = max(mx, cur - sums[bisect.bisect_left(sums, cur - k)])
                mxTotal = max(mxTotal, mx)
        return mxTotal

# 0363 Hard 363 Max Sum of Rectangle No Larger Than K

In [ ]:
"""
Time: O(min(M,N)^2 * max(M,N) * Log(max(M,N)))
Space: O(MN), can reduce to O(max(M,N)) by not using a new 2D array "matrixRotated".
Just change the iteration in `maxSumSubmatrix()`.

This is the implementation of the offical solution 1 and 2.
"""
from sortedcontainers import SortedSet

class Solution(object):
    def maxSumSubmatrix(self, matrix, k):
        if len(matrix)>len(matrix[0]): matrix = self.rotate(matrix)
        
        ans = float('-inf')
        M = len(matrix)
        N = len(matrix[0])
        
        for start in xrange(M):
            rowSum = [0]*N #row sum of rows from matrix[start] to row
            for row in matrix[start:]:
                for i, n in enumerate(row): rowSum[i] += n
                ans = max(ans, self.maxSumRow(rowSum, k))
                if ans==k: return ans
        return ans
    
    def maxSumRow(self, row, k):
        ans = float('-inf')
        total = 0
        
        ss = SortedSet()
        ss.add(0)
        
        for n in row:
            total += n
            i = ss.bisect_left(total-k)
            if i<len(ss):
                x = ss[i]
                ans = max(ans, total-x)
            ss.add(total)
        return ans
            
    def rotate(self, matrix):
        M = len(matrix)
        N = len(matrix[0])
        matrixRotated = [[0]*M for _ in xrange(N)]
        
        for i in xrange(M):
            for j in xrange(N):
                matrixRotated[j][i] = matrix[i][j]
        
        return matrixRotated
            

# 0381  Hard  381. Insert Delete GetRandom O

In [ ]:
class RandomizedCollection:

    def __init__(self):
        self.arr, self.pos = [], collections.defaultdict(set)
    def insert(self, val):
        out = val not in self.pos
        self.arr.append(val)
        self.pos[val].add(len(self.arr) - 1)
        return out

    def remove(self, val):
        if val in self.pos:
            if self.arr[-1] != val:
                x, y = self.pos[val].pop(), self.arr[-1]
                self.pos[y].discard(len(self.arr) - 1)
                self.pos[y].add(x)
                self.arr[x] = y
            else:
                self.pos[val].discard(len(self.arr) - 1)
            self.arr.pop()
            if not self.pos[val]:
                self.pos.pop(val)
            return True 
        return False

    def getRandom(self):
        return random.choice(self.arr)

# 0391  Hard  391. Perfect Rectangle

In [ ]:
class Solution:
    def isRectangleCover(self, rectangles):
        cnt = collections.Counter()
        for x1, y1, x2, y2 in rectangles:
            cnt[(x1, y1)] += 1
            cnt[(x1, y2)] += 1
            cnt[(x2, y2)] += 1
            cnt[(x2, y1)] += 1
        x1, y1, x2, y2 = min([r[:2] for r in rectangles]) + max(r[-2:] for r in rectangles)
        for x, y in ((x1, y1), (x1, y2), (x2, y2), (x2, y1)):
            if cnt[(x, y)] != 1: return False
            cnt.pop((x, y))
        return all(cnt[k] in (2, 4) for k in cnt) and sum((x2 - x1) * (y2 - y1) for x1, y1, x2, y2 in rectangles) == (x2 - x1) * (y2 - y1)

# 0391 Hard 391 Perfect Rectangle

In [ ]:
# Time:  O(n)
# Space: O(n)

from collections import defaultdict

class Solution(object):
    def isRectangleCover(self, rectangles):
        """
        :type rectangles: List[List[int]]
        :rtype: bool
        """
        left = min(rec[0] for rec in rectangles)
        bottom = min(rec[1] for rec in rectangles)
        right = max(rec[2] for rec in rectangles)
        top = max(rec[3] for rec in rectangles)

        points = defaultdict(int)
        for l, b, r, t in rectangles:
            for p, q in zip(((l, b), (r, b), (l, t), (r, t)), (1, 2, 4, 8)):
                if points[p] & q:
                    return False
                points[p] |= q

        for px, py in points:
            if left < px < right or bottom < py < top:
                if points[(px, py)] not in (3, 5, 10, 12, 15):
                    return False

        return True

# 0403  Hard  403. Frog Jump

In [ ]:
class Solution:
    def canCross(self, stones):
        memo, stones, target = {}, set(stones), stones[-1]
        def dfs(unit, last):
            if unit == target: return True
            if (unit, last) not in memo: 
                memo[(unit, last)] = any(dfs(unit + move, move) for move in (last - 1, last, last + 1) if move and unit + move in stones)
            return memo[(unit, last)]
        return dfs(1, 1) if 1 in stones else False

# 0403 Hard 403 Frog Jump

In [ ]:
# Time:  O(n^2)
# Space: O(n^2)

class Solution(object):
    def canCross(self, stones):
        """
        :type stones: List[int]
        :rtype: bool
        """
        if stones[1] != 1:
            return False

        last_jump_units = {s: set() for s in stones}
        last_jump_units[1].add(1)
        for s in stones[:-1]:
            for j in last_jump_units[s]:
                for k in (j-1, j, j+1):
                    if k > 0 and s+k in last_jump_units:
                        last_jump_units[s+k].add(k)
        return bool(last_jump_units[stones[-1]])

# 0407  Hard  407. Trapping Rain Water II

In [ ]:
class Solution:
    def trapRainWater(self, heightMap):
        m, n, heap, trapped = len(heightMap), len(heightMap and heightMap[0]), [], 0
        for i in range(m):
            for j in range(n):
                if i in {0, m - 1} or j in {0, n - 1}:
                    heapq.heappush(heap, (heightMap[i][j], i, j))
                    heightMap[i][j] = -1          
        while heap:
            h, i, j = heapq.heappop(heap)
            for x, y in ((i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)):
                if 0 < x < m - 1 and 0 < y < n - 1 and heightMap[x][y] != -1:
                    trapped += max(h - heightMap[x][y], 0)
                    heapq.heappush(heap, (max(heightMap[x][y], h), x, y))
                    heightMap[x][y] = -1
        return trapped

# 0407 Hard 407 Trapping Rain Water II

In [ ]:
class Solution(object):
    def trapRainWater(self, heightMap):
        pq = []
        N = len(heightMap)
        M = len(heightMap[0])
        visited = set()
        ans = 0
        curr = float('-inf')
        
        for i in xrange(N):
            for j in xrange(M):
                if i==0 or i==N-1 or j==0 or j==M-1:
                    heapq.heappush(pq, (heightMap[i][j], i, j))
                
        while pq:
            h, i, j = heapq.heappop(pq)
            if (i, j) in visited: continue
            visited.add((i, j))
            
            if h>curr: curr = h
            ans += (curr-h)
            
            for iNext, jNext in [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]:
                if iNext<0 or iNext>=N or jNext<0 or jNext>=M: continue
                if (iNext, jNext) in visited: continue
                heapq.heappush(pq, (heightMap[iNext][jNext], iNext, jNext))
        
        return ans

# 0410  Hard  410. Split Array Largest Sum

In [ ]:
class Solution:
    def splitArray(self, nums, m):
        def valid(mid):
            cnt = sm = 0
            for num in nums:
                sm += num
                if sm > mid:
                    cnt += 1
                    if cnt>= m: return False
                    sm = num
            return True
        l, h = max(nums), sum(nums)
        while l < h:
            mid = (l + h) // 2
            if valid(mid):
                h = mid
            else:
                l = mid + 1
        return l

# 0410 Hard 410 Split Array Largest Sum

In [ ]:
"""
TLE
dp[i][k] := min of the largest sum among k subarrays from nums[:i]
"""
class Solution(object):
    def splitArray(self, nums, K):
        N = len(nums)
        
        dp = [[float('inf') for _ in xrange(K+1)] for _ in xrange(N+1)]
        dp[0][0] = 0
        
        for i in xrange(1, N+1):
            for k in xrange(1, min(i, K)+1):
                for j in xrange(k, i+1):
                    dp[i][k] = min(dp[i][k], max(dp[j-1][k-1], sum(nums[j-1:i])))
        
        return dp[N][K]


"""
TLE
dp[i][k] := min of the largest sum among k subarrays from nums[:i]
s[i][j] := sum of nums[i:j+1]
"""
class Solution(object):
    def splitArray(self, nums, K):
        N = len(nums)

        s = [[0 for _ in xrange(N)] for _ in xrange(N)]
        for i in xrange(N): s[i][i] = nums[i]
        for l in xrange(2, N):
            for i in xrange(N):
                j = i+l-1
                if j>=N: continue
                s[i][j] = s[i+1][j-1]+nums[i]+nums[j]

        dp = [[float('inf') for _ in xrange(K+1)] for _ in xrange(N+1)]
        dp[0][0] = 0
        
        for i in xrange(1, N+1):
            for k in xrange(1, min(i, K)+1):
                for j in xrange(k, i+1):
                    dp[i][k] = min(dp[i][k], max(dp[j-1][k-1], s[j-1][i-1]))
        
        return dp[N][K]

# 0411  Hard  411. Minimum Unique Word Abbreviation

In [ ]:
class Solution(object):
    def extract_number(self, j, abbr, M):
        num = 0
        while j < M and abbr[j].isdigit():
            num, j = num*10 + int(abbr[j]), j+1
        return num, j
    
    def valid(self, word, abbr):
        i,j,N, M = 0,0,len(word), len(abbr)
        while i < N and j < M:
            if abbr[j].isalpha() and abbr[j] != word[i]:
                return False
            elif abbr[j].isalpha() and abbr[j] == word[i]:
                i,j = i+1,j+1
            elif abbr[j].isdigit():
                if abbr[j] == '0':
                    return False
                num, j = self.extract_number(j, abbr, M)
                i = i+num
        return (i==N and j == M)
        
    def process_solution(self, so_far):
        csofar, i, cnt = [], 0, 0
        while i < len(so_far):
            if so_far[i].isalpha():
                csofar.append(so_far[i])
                i, cnt = i+1, cnt+1
            else:
                num = 0
                while i < len(so_far) and so_far[i].isdigit():
                    num, i = num+1, i+1
                cnt = cnt + 1
                csofar.append(str(num))
        return "".join(csofar), cnt
    
    def test(self, abbr, dictionary):
        for wrd in dictionary:
            if self.valid(wrd, abbr):
                return False
        return True
    
    def helper(self, word, so_far, i, dictionary):
        if i == len(word):
            abbr, cnt = self.process_solution(so_far)
            if cnt < self.result_len and self.test(abbr, dictionary):
                self.result, self.result_len = abbr, cnt
            return
        else:
            so_far.append("1")
            self.helper(word, so_far, i+1, dictionary)
            so_far.pop()
            so_far.append(word[i])
            self.helper(word, so_far, i+1, dictionary)
            so_far.pop()

    def minAbbreviation(self, target, dictionary):
        """
        :type target: str
        :type dictionary: List[str]
        :rtype: str
        """
        
        # Remove those words which can never be an abbreviation for target.
        # This preprocessing will help us save time.
        filtered_dictionary = []
        for wrd in dictionary:
            if len(wrd) != len(target):
                continue
            filtered_dictionary.append(wrd)
        dictionary = filtered_dictionary
        if len(dictionary) == 0:
            return str(len(target))
            
        self.result_len = len(target)+1
        self.result, so_far, i = target, [], 0
        self.helper(target, so_far, i, dictionary)        
        return self.result

# 0411 Hard 411 Minimum Unique Word Abbreviation

In [ ]:
# Time:  O((d + n) * 2^n)
# Space: O(d)

# optimized from Solution2
class Solution(object):
    def minAbbreviation(self, target, dictionary):
        """
        :type target: str
        :type dictionary: List[str]
        :rtype: str
        """
        def bits_to_abbr_len(targets, bits):
            total = 0
            pre = 0
            for i in xrange(len(target)):
                if bits & 1:
                    if i - pre > 0:
                        total += len(str(i - pre))
                    pre = i + 1
                    total += 1
                elif i == len(target) - 1:
                    total += len(str(i - pre + 1))
                bits >>= 1
            return total

        def bits_to_abbr(targets, bits):
            abbr = []
            pre = 0
            for i in xrange(len(target)):
                if bits & 1:
                    if i - pre > 0:
                        abbr.append(str(i - pre))
                    pre = i + 1
                    abbr.append(target[i])
                elif i == len(target) - 1:
                    abbr.append(str(i - pre + 1))
                bits >>= 1
            return "".join(abbr)
  
        diffs = []
        for word in dictionary:
            if len(word) != len(target):
                continue
            diffs.append(sum(2**i for i, c in enumerate(word) if target[i] != c))

        if not diffs:
            return str(len(target))

        result = 2**len(target)-1
        for mask in xrange(2**len(target)):
            if all(d & mask for d in diffs) and bits_to_abbr_len(target, mask) < bits_to_abbr_len(target, result):
                result = mask
        return bits_to_abbr(target, result)
    

# Time:  O((d + n) * 2^n)
# Space: O(d + n)
class Solution2(object):
    def minAbbreviation(self, target, dictionary):
        """
        :type target: str
        :type dictionary: List[str]
        :rtype: str
        """
        def bits_to_abbr(targets, bits):
            abbr = []
            pre = 0
            for i in xrange(len(target)):
                if bits & 1:
                    if i - pre > 0:
                        abbr.append(str(i - pre))
                    pre = i + 1
                    abbr.append(target[i])
                elif i == len(target) - 1:
                    abbr.append(str(i - pre + 1))
                bits >>= 1
            return "".join(abbr)
  
        diffs = []
        for word in dictionary:
            if len(word) != len(target):
                continue
            diffs.append(sum(2**i for i, c in enumerate(word) if target[i] != c))

        if not diffs:
            return str(len(target))

        result = target
        for mask in xrange(2**len(target)):
            abbr = bits_to_abbr(target, mask)
            if all(d & mask for d in diffs) and len(abbr) < len(result):
                result = abbr
        return result

# 0420  Hard  420. Strong Password Checker

In [ ]:
class Solution(object):
    def strongPasswordChecker(self, s):
        """
        :type s: str
        :rtype: int
        """
        missing_type = 3
        if any('a' <= c <= 'z' for c in s): missing_type -= 1
        if any('A' <= c <= 'Z' for c in s): missing_type -= 1
        if any(c.isdigit() for c in s): missing_type -= 1

        change = 0
        one = two = 0
        p = 2
        while p < len(s):
            if s[p] == s[p-1] == s[p-2]:
                length = 2
                while p < len(s) and s[p] == s[p-1]:
                    length += 1
                    p += 1
                    
                change += length // 3
                if length % 3 == 0: one += 1
                elif length % 3 == 1: two += 1
            else:
                p += 1
        
        if len(s) < 6:
            return max(missing_type, 6 - len(s))
        elif len(s) <= 20:
            return max(missing_type, change)
        else:
            delete = len(s) - 20
            
            change -= min(delete, one)
            change -= min(max(delete - one, 0), two * 2) // 2
            change -= max(delete - one - 2 * two, 0) // 3
                
            return delete + max(missing_type, change)

# 0420 Hard 420 Strong Password Checker

In [ ]:
# Time:  O(n)
# Space: O(1)

class Solution(object):
    def strongPasswordChecker(self, s):
        """
        :type s: str
        :rtype: int
        """
        missing_type_cnt = 3
        if any('a' <= c <= 'z' for c in s):
            missing_type_cnt -= 1
        if any('A' <= c <= 'Z' for c in s):
            missing_type_cnt -= 1
        if any(c.isdigit() for c in s):
            missing_type_cnt -= 1

        total_change_cnt = 0
        one_change_cnt, two_change_cnt, three_change_cnt = 0, 0, 0
        i = 2
        while i < len(s):
            if s[i] == s[i-1] == s[i-2]:
                length = 2
                while i < len(s) and s[i] == s[i-1]:
                    length += 1
                    i += 1

                total_change_cnt += length / 3
                if length % 3 == 0:
                    one_change_cnt += 1
                elif length % 3 == 1:
                    two_change_cnt += 1
                else:
                    three_change_cnt += 1
            else:
                i += 1

        if len(s) < 6:
            return max(missing_type_cnt, 6 - len(s))
        elif len(s) <= 20:
            return max(missing_type_cnt, total_change_cnt)
        else:
            delete_cnt = len(s) - 20

            total_change_cnt -= min(delete_cnt, one_change_cnt * 1) / 1
            total_change_cnt -= min(max(delete_cnt - one_change_cnt, 0), two_change_cnt * 2) / 2
            total_change_cnt -= min(max(delete_cnt - one_change_cnt - 2 * two_change_cnt, 0), three_change_cnt * 3) / 3

            return delete_cnt + max(missing_type_cnt, total_change_cnt)

# 0425  Hard  425. Word Squares

In [ ]:
class Solution:
    def wordSquares(self, words):
        pref, res = collections.defaultdict(set), []
        for w in words:
            for i in range(len(w)):
                pref[w[:i + 1]].add(w)
        def dfs(i, arr):
            if i == len(arr[0]):
                res.append(arr)
            else:
                for w in pref["".join(row[i] for row in arr)]:
                    dfs(i + 1, arr + [w])   
        for w in words:
            dfs(1, [w])
        return res

# 0425 Hard 425 Word Squares

In [ ]:
# Time:  O(n^2 * n!)
# Space: O(n^2)

class TrieNode(object):
    def __init__(self):
        self.indices = []
        self.children = [None] * 26

    def insert(self, words, i):
        cur = self
        for c in words[i]:
            if not cur.children[ord(c)-ord('a')]:
                cur.children[ord(c)-ord('a')] = TrieNode()
            cur = cur.children[ord(c)-ord('a')]
            cur.indices.append(i)


class Solution(object):
    def wordSquares(self, words):
        """
        :type words: List[str]
        :rtype: List[List[str]]
        """
        result = []

        trie = TrieNode()
        for i in xrange(len(words)):
            trie.insert(words, i)

        curr = []
        for s in words:
            curr.append(s)
            self.wordSquaresHelper(words, trie, curr, result)
            curr.pop()

        return result

    def wordSquaresHelper(self, words, trie, curr, result):
        if len(curr) >= len(words[0]):
            return result.append(list(curr))

        node = trie
        for s in curr:
            node = node.children[ord(s[len(curr)]) - ord('a')]
            if not node:
                return

        for i in node.indices:
            curr.append(words[i])
            self.wordSquaresHelper(words, trie, curr, result)
            curr.pop()

# 0428  Hard  428. Serialize and Deserialize N ary Tree

In [ ]:
class Codec:

    def serialize(self, root):
        arr = []
        def dfs(node):
            if not node: return
            arr.append(str(node.val))
            for child in node.children:
                dfs(child)
            arr.append("#")
        dfs(root)
        return " ".join(arr)

    def deserialize(self, data):
        if not data: return None
        data = collections.deque(data.split(" "))
        root = Node(int(data.popleft()), [])
        q = [root]
        while data:
            val = data.popleft()
            if val == "#":
                q.pop()
            else:
                new = Node(int(val), [])
                q[-1].children.append(new)
                q.append(new)
        return root

# 0431  Hard  431. Encode N ary Tree to Binary Tree

In [ ]:
"""
# Definition for a Node.
class Node(object):
    def __init__(self, val, children):
        self.val = val
        self.children = children
"""
"""
# Definition for a binary tree node.
class TreeNode(object):
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None
"""
class Codec:

    def encode(self, root):
        """Encodes an n-ary tree to a binary tree.
        
        :type root: Node
        :rtype: TreeNode
        """
        if not root:
            return None

        binary = TreeNode(root.val)                 # create a binary root
        if not root.children:
            return binary

        binary.left = self.encode(root.children[0]) # left child of binary is the encoding of all n-ary children,
        node = binary.left                          #     starting with the first child.
        for child in root.children[1:]:             # other children of n-ary root are right child of previous child
            node.right = self.encode(child)
            node = node.right

        return binary

    def decode(self, data):
        """Decodes your binary tree to an n-ary tree.
        
        :type data: TreeNode
        :rtype: Node
        """
        if not data:
            return None

        nary = Node(data.val, [])                   # create n-ary root
        node = data.left                            # move to first child of n-ary root
        while node:                                 # while more children of n-ary root
            nary.children.append(self.decode(node)) # append to list
            node = node.right                       # and move to next child
            
        return nary

# Your Codec object will be instantiated and called as such:
# codec = Codec()
# codec.decode(codec.encode(root))

# 0432  Hard  432. All O`one Data Structure

In [ ]:
class Node:
    def __init__(self, key, value):
        self.val = value
        self.key = key
        self.next = None
        self.pre = None
class AllOne:

    def __init__(self):
        self.first = {}
        self.last = {}
        self.keys = {}
        self.head = Node(-1, -1)
        self.tail = Node(-1, -1)
        self.head.next = self.tail
        self.tail.pre = self.head
        
    def add(self, prev, node):
        node.pre = prev
        node.next = prev.next
        node.pre.next = node.next.pre = node
        
    def remove(self, node):
        node.pre.next = node.next
        node.next.pre = node.pre
        
    def process(self, node):
        if self.last[node.val] == node and node.pre.val != node.val:
            self.first.pop(node.val)
            self.last.pop(node.val)
        elif self.first[node.val] == node:
            self.first[node.val] = node.next
        elif self.last[node.val] == node:
            self.last[node.val] = node.pre
            
    def process2(self, node, prev, key, d):
        if key in self.keys:
            if node.val + d in self.last:
                self.add(self.last[node.val + d], node)
            elif node.val in self.last:
                self.add(self.last[node.val], node)
            else:
                self.add(prev, node)
        elif 1 in self.last:
            node = Node(key, 0)
            self.add(self.last[1], node)
        else:
            node = Node(key, 0)
            self.add(self.head, node)
        node.val += d
        self.last[node.val] = node
        if node.val not in self.first:
            self.first[node.val] = node
        if key not in self.keys:
            self.keys[key] = node
                
    def inc(self, key):
        if key in self.keys:
            node = self.keys[key]
            prev = node.pre
            self.process(node)
            self.remove(node)
            self.process2(node, prev, key, 1)
        else:
            self.process2(None, None, key, 1)
            
    def dec(self, key):
        if key in self.keys:
            node = self.keys[key]
            prev = node.pre
            self.process(node)
            self.remove(node)
            if node.val != 1:
                self.process2(node, prev, key, -1)
            else:
                self.keys.pop(key)
            
    def getMaxKey(self):
        return self.tail.pre.key if self.tail.pre != self.head else ""
    def getMinKey(self):
        return self.head.next.key if self.head.next != self.tail else ""

# 0440  Hard  440. K th Smallest in Lexicographical Order

In [ ]:
class Solution:
    def findKthNumber(self, n, k):
        def calSteps(n1, n2):
            steps = 0
            while n1 <= n:
                steps += min(n + 1, n2) - n1
                n1 *= 10
                n2 *= 10
            return steps
        cur = 1
        k -= 1
        while k:
            steps = calSteps(cur, cur + 1)
            if steps <= k:
                cur += 1
                k -= steps
            else:
                cur *= 10
                k -= 1
        return cur

# 0446  Hard  446. Arithmetic Slices II  Subsequence

In [ ]:
class Solution:
    def numberOfArithmeticSlices(self, A):
        dp, res = collections.defaultdict(dict), 0
        for j in range(len(A)):
            for i in range(j):
                dp[j][A[j] - A[i]] = dp[j].get(A[j] - A[i], 0) + dp[i].get(A[j] - A[i], 1) 
                if A[j] - A[i] in dp[i]: res, dp[j][A[j] - A[i]] = res + dp[i][A[j] - A[i]], dp[j][A[j] - A[i]] + 1
        return res           

# 0458  Hard  458. Poor Pigs

In [ ]:
class Solution(object):
    def poorPigs(self, buckets, minutesToDie, minutesToTest):
        """
        :type buckets: int
        :type minutesToDie: int
        :type minutesToTest: int
        :rtype: int
        """
        pigs = 0
        while (minutesToTest / minutesToDie + 1) ** pigs < buckets: pigs += 1
        return pigs

# 0458 Hard 458. Poor Pigs

In [ ]:
class Solution(object):
    def poorPigs(self, buckets, minutesToDie, minutesToTest):
        """
        :type buckets: int
        :type minutesToDie: int
        :type minutesToTest: int
        :rtype: int
        """
        # https://leetcode.com/problems/poor-pigs/discuss/94266/Another-explanation-and-solution
        pigs = 0
        while (minutesToTest / minutesToDie + 1) ** pigs < buckets:
            pigs += 1
        return pigs

# 0458 Hard 458 Poor Pigs

In [ ]:
# Time:  O(1)
# Space: O(1)

import math


class Solution(object):
    def poorPigs(self, buckets, minutesToDie, minutesToTest):
        """
        :type buckets: int
        :type minutesToDie: int
        :type minutesToTest: int
        :rtype: int
        """
        return int(math.ceil(math.log(buckets) / math.log(minutesToTest / minutesToDie + 1)))

# 0460  Hard  460. LFU Cache

In [ ]:
class Node:
    def __init__(self, k, v, f):
        self.key = k
        self.val = v
        self.freq = f
        self.next = self.pre = None
class LFUCache:

    def __init__(self, capacity):
        self.max = capacity
        self.cache = 0
        self.freqLast = {}
        self.Nodes = {}
        self.head = self.tail = Node("#", "#", "#")
        self.head.next = self.tail
        self.tail.pre = self.head

    def changeFreq(self, key):
        node, f = self.Nodes[key], self.Nodes[key].freq
        if self.freqLast[f] == node:
            if node.pre.freq == f:
                self.freqLast[f] = node.pre
            else:
                self.freqLast.pop(f)
        if f + 1 in self.freqLast:
            node.pre.next = node.next
            node.next.pre = node.pre
            node.pre = self.freqLast[f + 1]
            node.next = node.pre.next
        elif f in self.freqLast:
            node.pre.next = node.next
            node.next.pre = node.pre
            node.pre = self.freqLast[f]
            node.next = node.pre.next
        node.pre.next = node.next.pre = node
        self.freqLast[f + 1] = node
        node.freq += 1
        
    def removeFirst(self):
        node, f = self.head.next, self.head.next.freq
        node.pre.next = node.next
        node.next.pre = node.pre
        self.Nodes.pop(node.key)
        if self.freqLast[f] == node:
            self.freqLast.pop(f)
        self.cache -= 1
    
    def get(self, key):
        if key in self.Nodes:
            self.changeFreq(key)
            return self.Nodes[key].val
        return -1
    def put(self, key, value):
        if key in self.Nodes:
            self.changeFreq(key)
            self.Nodes[key].val = value
        elif self.max:
            if self.cache == self.max:
                self.removeFirst()
            self.cache += 1
            new = Node(key, value, 1)
            if 1 in self.freqLast:
                new.pre = self.freqLast[1]
            else:
                new.pre = self.head
            new.next = new.pre.next
            new.pre.next = new.next.pre = new
            self.freqLast[1] = self.Nodes[key] = new

# 0460 Hard 460 LFU Cache

In [ ]:
# Time:  O(1), per operation
# Space: O(k), k is the capacity of cache

import collections


# using OrderedDict
class LFUCache(object):

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.__capa = capacity
        self.__size = 0
        self.__min_freq = float("inf")
        self.__freq_to_nodes = collections.defaultdict(collections.OrderedDict)
        self.__key_to_freq = {}

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key not in self.__key_to_freq:
            return -1
        value = self.__freq_to_nodes[self.__key_to_freq[key]][key]
        self.__update(key, value)
        return value

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        if self.__capa <= 0:
            return

        if key not in self.__key_to_freq and self.__size == self.__capa:
            del self.__key_to_freq[self.__freq_to_nodes[self.__min_freq].popitem(last=False)[0]]
            if not self.__freq_to_nodes[self.__min_freq]:
                del self.__freq_to_nodes[self.__min_freq]
            self.__size -= 1
        self.__update(key, value)
        
    def __update(self, key, value):
        freq = 0
        if key in self.__key_to_freq:
            freq = self.__key_to_freq[key]
            del self.__freq_to_nodes[freq][key]
            if not self.__freq_to_nodes[freq]:
                del self.__freq_to_nodes[freq]
                if self.__min_freq == freq:
                    self.__min_freq += 1
            self.__size -= 1

        freq += 1
        self.__min_freq = min(self.__min_freq, freq)
        self.__key_to_freq[key] = freq
        self.__freq_to_nodes[freq][key] = value
        self.__size += 1


# Time:  O(1), per operation
# Space: O(k), k is the capacity of cache
import collections


class ListNode(object):
    def __init__(self, key, value, freq):
        self.key = key
        self.val = value
        self.freq = freq
        self.next = None
        self.prev = None


class LinkedList(object):
    def __init__(self):
        self.head = None
        self.tail = None

    def append(self, node):
        node.next, node.prev = None, None  # avoid dirty node
        if self.head is None:
            self.head = node
        else:
            self.tail.next = node
            node.prev = self.tail
        self.tail = node

    def delete(self, node):
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next
        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev
        node.next, node.prev = None, None  # make node clean


class LFUCache2(object):

    def __init__(self, capacity):
        """
        :type capacity: int
        """
        self.__capa = capacity
        self.__size = 0
        self.__min_freq = float("inf")
        self.__freq_to_nodes = collections.defaultdict(LinkedList)
        self.__key_to_node = {}

    def get(self, key):
        """
        :type key: int
        :rtype: int
        """
        if key not in self.__key_to_node:
            return -1
        value = self.__key_to_node[key].val
        self.__update(key, value)
        return value

    def put(self, key, value):
        """
        :type key: int
        :type value: int
        :rtype: void
        """
        if self.__capa <= 0:
            return

        if key not in self.__key_to_node and self.__size == self.__capa:
            del self.__key_to_node[self.__freq_to_nodes[self.__min_freq].head.key]
            self.__freq_to_nodes[self.__min_freq].delete(self.__freq_to_nodes[self.__min_freq].head)
            if not self.__freq_to_nodes[self.__min_freq].head:
                del self.__freq_to_nodes[self.__min_freq]
            self.__size -= 1
        self.__update(key, value)
        
    def __update(self, key, value):
        freq = 0
        if key in self.__key_to_node:
            old_node = self.__key_to_node[key]
            freq = old_node.freq
            self.__freq_to_nodes[freq].delete(old_node)
            if not self.__freq_to_nodes[freq].head:
                del self.__freq_to_nodes[freq]
                if self.__min_freq == freq:
                    self.__min_freq += 1
            self.__size -= 1

        freq += 1
        self.__min_freq = min(self.__min_freq, freq)
        self.__key_to_node[key] = ListNode(key, value, freq)
        self.__freq_to_nodes[freq].append(self.__key_to_node[key])
        self.__size += 1

# 0465  Hard  465. Optimal Account Balancing

In [ ]:
class Solution:
    def minTransfers(self, transactions: List[List[int]]) -> int:
        def remove_one_zero_clique(non_zero):
            n = len(non_zero)
            q = collections.deque()            
            # q store ([index set], sum of set)
            q.append(([0], non_zero[0]))
            min_zero_set = None

            while q:
                cur_set, cur_sum = q.popleft()
                if cur_sum == 0:
                    min_zero_set = cur_set
                    break
                for j in range(cur_set[-1] + 1, n):
                    q.append((cur_set + [j], cur_sum + non_zero[j]))
            
            min_zero_set = set(min_zero_set)
            return [non_zero[i] for i in range(n) if i not in min_zero_set]
        
        
        bal = collections.defaultdict(int)
        for t in transactions:
            bal[t[0]] -= t[2]
            bal[t[1]] += t[2]
        non_zero = [bal[k] for k in bal if bal[k] != 0]
        
        bal_cnt = len(non_zero)
        while len(non_zero) > 0:
            non_zero = remove_one_zero_clique(non_zero)
            bal_cnt -= 1
        return bal_cnt